In [ ]:
!pip install transformers
!pip install datasets --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 14.8 MB/s 
     |████████████████████████████████| 7.6 MB 54.4 MB/s 
     |████████████████████████████████| 182 kB 57.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 13.8 MB/s 
     |████████████████████████████████| 115 kB 63.4 MB/s 
     |████████████████████████████████| 212 kB 72.1 MB/s 
     |████████████████████████████████| 127 kB 66.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from transformers import TFAutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
# model_checkpoint = "bert-base-cased"
model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


In [ ]:
model(model.dummy_inputs)  # Build the model
model.summary()

Model: "tf_distil_bert_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 vocab_transform (Dense)     multiple                  590592    
                                                                 
 vocab_layer_norm (LayerNorm  multiple                 1536      
 alization)                                                      
                                                                 
 vocab_projector (TFDistilBe  multiple                 23866170  
 rtLMHead)                                                       
                                                                 
Total params: 66,985,530
Trainable params: 66,985,530
Non-trainable params: 0
__________________________

In [ ]:
text = "This is a great [MASK], I prefer [MASK]."


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import numpy as np
import tensorflow as tf

inputs = tokenizer(text, return_tensors="np")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
# We negate the array before argsort to get the largest, not the smallest, logits
top_5_tokens = np.argsort(-mask_token_logits)[:5].tolist()

for token in top_5_tokens:
    print(f">>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}")

>>> This is a great idea, I prefer idea.
>>> This is a great place, I prefer place.
>>> This is a great deal, I prefer deal.
>>> This is a great job, I prefer job.
>>> This is a great plan, I prefer plan.


In [ ]:
## TESTING
text1 = "[MASK] celebrated the victory of Obama in 2012."
text2 = "[MASK] decided to remove their mask during the press conference."

inputs = tokenizer(text1, return_tensors="np")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
# We negate the array before argsort to get the largest, not the smallest, logits
all_tokens = np.argsort(-mask_token_logits).tolist()

word_probability_dict = {}
for index, val in enumerate(mask_token_logits):
  word = tokenizer.decode([index])
  word_probability_dict[word] = val.numpy()

word_list = []
print("Democrats prob. score:", word_probability_dict['Democrats'])
print("Republicans prob. score:", word_probability_dict['Republicans'])
print("Trump prob. score:", word_probability_dict['Trump'])
# print("Biden prob. score:", word_probability_dict['biden'])

In [ ]:
# LOAD DATASET
# import pandas as pd
# df = pd.read_parquet('all_2020_US_Election_News.parquet', engine='pyarrow')

In [ ]:
# MOUNT DRIVE
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
%cd /content/drive/My Drive/Independent Study/
df = pd.read_parquet('all_2020_US_Election_News.parquet', engine='pyarrow')

# LOADING DATASET IN HUGGING FACE FORM
# from datasets import load_dataset
# df = load_dataset("parquet", data_files={'all_2020_US_Election_News.parquet'})

/content/drive/My Drive/Independent Study


In [ ]:
# Saving the base model
model.save_weights('./checkpoints/my_base_BERT_model')

In [ ]:
df.iloc[3]['text']

'Senate showdown over pandemic relief stalls as Pelosi preps Democratic bill The Senate\'s efforts to advance a bipartisan proposal have stalled.\n\nThe Senate\'s second attempt to advance a massive $2 trillion bill to provide economic relief amid the COVID-19 pandemic failed Monday afternoon, demonstrating an inability to strike a bipartisan agreement on the Coronavirus Aid, Relief, and Economic Security Act.\n\nThe vote failed 49-46, falling well short of the 60 votes needed to advance to debate on the bill, which represents the third phase of congressional aid to respond to the crisis.\n\nAfter the vote, Senate Majority Mitch McConnell warned that the Senate might not be able to pass a bill until Friday or Saturday, blasting Democrats for "mindless obstruction."\n\n"While the public is waiting for us to act, while people losing their jobs, losing their income and shutting down the economy which we\'ve had to do to deal with this public health crisis, they\'re fiddling around with Se

In [ ]:
# for index, row in df.iterrows():
#   if row.publication == 'ABC News':
#     row['leaning'] = 'L'
#   break
def label_leaning(row):
  if row['publication'] == 'ABC News' or row['publication'] == 'CBS News' or row['publication'] == 'CNN' or row['publication'] == 'MSNBC' or row['publication'] == 'The Washington Post' or row['publication'] == 'Al Jazeera English' or row['publication'] == 'Google News' or row['publication'] == 'Buzzfeed':
    return 'L'
  elif row['publication'] in ['Fox News', 'Breitbart News', 'The Washington Times']:
    return 'R'
  return 'C'

df['leaning'] = df.apply (lambda row: label_leaning(row), axis=1)


In [ ]:
df.iloc[0]

Unnamed: 0                                                     0
id                                                             1
author                                      The Associated Press
date                                         2020-03-23 19:19:00
title          Review: 'The Glass Hotel a tragic tale for the...
publication                                             ABC News
text           The Glass Hotel, Alfred A. Knopf, by Emily St....
keywords       ['hotel', 'tragic', 'vancouver', 'end', 'revie...
summary        The Glass Hotel, Alfred A. Knopf, by Emily St....
leaning                                                        L
Name: 0, dtype: object

In [ ]:
print(df.size)

1443720


In [ ]:
df2 = pd.DataFrame().assign(text=df['text'], leaning=df['leaning'])
df2 = df2.dropna()

df2.size

277754

In [ ]:
grouped_df = df2.groupby('leaning')

In [ ]:
grouped_df = df2.groupby('leaning').apply(lambda a: a[:])

In [ ]:
dfs = [x for _, x in df2.groupby('leaning')]

In [ ]:
C_dataset, L_dataset, R_dataset = dfs[0], dfs[1], dfs[2]

In [ ]:
print(len(C_dataset), len(L_dataset), len(R_dataset))

30854 64752 43271


In [ ]:
C_dataset.to_csv (r'export_C_dataframe.csv', index = False, header=True)
L_dataset.to_csv (r'export_L_dataframe.csv', index = False, header=True)
R_dataset.to_csv (r'export_R_dataframe.csv', index = False, header=True)

In [ ]:
# LOADING L DATASET IN HUGGING FACE FORM
from datasets import load_dataset
%cd /content/drive/My Drive/Independent Study/

# dataset = load_dataset("csv", data_files={'export_L_dataframe.csv'})
dataset = load_dataset("csv", data_files={'export_R_dataframe.csv'})

/content/drive/My Drive/Independent Study


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-428bb13fb119c164/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset = dataset['train']

In [ ]:
# Range for export_L_dataframe 
# train = dataset.shuffle(seed=42).select(range(25000))
# test = dataset.shuffle(seed=42).select(range(25000, 50000))
# unsupervised = dataset.shuffle(seed=42).select(range(50000, len(dataset)))

# Range for export_R_dataframe 
train = dataset.shuffle(seed=42).select(range(20000))
test = dataset.shuffle(seed=42).select(range(20000, 40000))
unsupervised = dataset.shuffle(seed=42).select(range(40000, len(dataset)))

In [ ]:
# pip install datasets
my_dataset_dict = {
    "train": train,
    "test": test,
    "unsupervised": unsupervised
    }

In [ ]:
len(my_dataset_dict['unsupervised'])

3271

In [ ]:
my_dataset_dict

{'train': Dataset({
     features: ['text', 'leaning'],
     num_rows: 20000
 }), 'test': Dataset({
     features: ['text', 'leaning'],
     num_rows: 20000
 }), 'unsupervised': Dataset({
     features: ['text', 'leaning'],
     num_rows: 3271
 })}

In [ ]:
def tokenize_function(examples):
    result = tokenizer(examples["text"])
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

# my_dataset_dict = my_dataset_dict.dropna()
# my_dataset_dict = my_dataset_dict['train']['tr'].tolist()
# texts_encodings = tokenizer(texts, truncation=True, padding=True)

# Use batched=True to activate fast multithreading!
train_tokenized_datasets = my_dataset_dict['train'].map(
    tokenize_function, batched=True, remove_columns=["text", "leaning"]
)
test_tokenized_datasets = my_dataset_dict['test'].map(
    tokenize_function, batched=True, remove_columns=["text", "leaning"]
)
unsupervised_tokenized_datasets = my_dataset_dict['unsupervised'].map(
    tokenize_function, batched=True, remove_columns=["text", "leaning"]
)

  0%|          | 0/20 [00:00<?, ?ba/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (851 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
print(len(train_tokenized_datasets), len(test_tokenized_datasets), len(unsupervised_tokenized_datasets))

20000 20000 3271


In [ ]:
tokenized_dataset = {
     "train": train_tokenized_datasets,
    "test": test_tokenized_datasets,
    "unsupervised": unsupervised_tokenized_datasets
}

In [ ]:
tokenized_dataset

{'train': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids'],
     num_rows: 20000
 }), 'test': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids'],
     num_rows: 20000
 }), 'unsupervised': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids'],
     num_rows: 3271
 })}

In [ ]:
print(tokenizer.model_max_length)
chunk_size = 128

512


In [ ]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"'>>> Review {idx} length: {len(sample)}'")

'>>> Review 0 length: 320'
'>>> Review 1 length: 851'
'>>> Review 2 length: 580'


In [ ]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"'>>> Concatenated reviews length: {total_length}'")

'>>> Concatenated reviews length: 1751'


In [ ]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 87'


In [ ]:
def group_texts(examples):
    # Concatenate all texts
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # Compute length of concatenated texts
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the last chunk if it's smaller than chunk_size
    total_length = (total_length // chunk_size) * chunk_size
    # Split by chunks of max_len
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # Create a new labels column
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
tokenized_dataset

{'train': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids'],
     num_rows: 20000
 }), 'test': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids'],
     num_rows: 20000
 }), 'unsupervised': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids'],
     num_rows: 3271
 })}

In [ ]:
lm_train_datasets = tokenized_dataset['train'].map(group_texts, batched=True)
lm_test_datasets = tokenized_dataset['test'].map(group_texts, batched=True)
lm_unsupervised_datasets = tokenized_dataset['unsupervised'].map(group_texts, batched=True)

lm_datasets = {
     "train": lm_train_datasets,
    "test": lm_test_datasets,
    "unsupervised": lm_unsupervised_datasets
}

  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [ ]:
lm_datasets

{'train': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
     num_rows: 110389
 }), 'test': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
     num_rows: 110933
 }), 'unsupervised': Dataset({
     features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
     num_rows: 18083
 })}

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'sleepy joe biden just closed down his campaign for the day ( again ). wants to rest! he is a very low energy individual, and our country cannot make it in these exciting, but complex and competitive times, with a low energy president!!! donald j. trump ( @ realdonaldtrump ) september 24, 2020 trump is revisiting the insult that he used against former florida gov. jeb bush ( r ) during the 2016 republican presidential primary. i used to call jeb bush, low energy. but i mean, if you compare the two, jeb is extraordinarily energetic compared to sleepy'

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["labels"])

'sleepy joe biden just closed down his campaign for the day ( again ). wants to rest! he is a very low energy individual, and our country cannot make it in these exciting, but complex and competitive times, with a low energy president!!! donald j. trump ( @ realdonaldtrump ) september 24, 2020 trump is revisiting the insult that he used against former florida gov. jeb bush ( r ) during the 2016 republican presidential primary. i used to call jeb bush, low energy. but i mean, if you compare the two, jeb is extraordinarily energetic compared to sleepy'

## **Fine-tuning DistilBERT with the Trainer API**


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



'>>> [CLS] president donald trump [MASK] his rival former [MASK] president joe biden [MASK] hosting zero public events on thursday. sleepy [MASK] biden just closed [MASK] his [MASK] for the day ( again ), trump wrote on twitter. wants to rest [MASK] [MASK] biden campaign called a [MASK] for reporters awaiting any activity from the campaign on thursday, as the former vice president is expected to focus on preparation for the debate. trump mocked biden for his [MASK] of energy on the campaign trail. he is a very low energy individual, [MASK] our country cannot make it [MASK] [MASK] exciting, but complex and competitive times, with a [MASK] energy president!!! he wrote.'

'>>> sleepy joe biden just closed down his campaign for the day ( again ). wants to rest! he is a very low energy individual, [MASK] our country cannot make [MASK] in these [MASK], but complex and competitive times [MASK] with a low energy president!! [MASK] donald j. trump ( @ [MASK]donald [MASK]p ) september 24, 2020 

In [ ]:
import collections
import numpy as np

from transformers.data.data_collator import tf_default_data_collator

wwm_probability = 0.2

def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # Create a map between words and corresponding token indices
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # Randomly mask words
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id

    return tf_default_data_collator(features)

In [ ]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] president donald trump mocked his rival former [MASK] president joe biden for [MASK] zero public events on thursday. sleepy [MASK] biden just closed down his campaign for the [MASK] ( again ), trump wrote on twitter. wants to rest! the biden campaign called a [MASK] for reporters awaiting any activity [MASK] the campaign [MASK] thursday, as the former vice president is expected to focus on preparation for the debate. trump mocked biden [MASK] [MASK] lack of [MASK] on the campaign trail. he is a very low energy individual, and our [MASK] cannot make it [MASK] these exciting, but complex and competitive times [MASK] [MASK] a low energy president!!! he wrote.'

'>>> [MASK] joe biden just closed [MASK] his campaign for [MASK] day [MASK] again ). wants to [MASK]! he is [MASK] [MASK] low energy individual [MASK] and our [MASK] [MASK] [MASK] it [MASK] these exciting, but complex and competitive times, [MASK] a low energy president [MASK]!! donald [MASK]. trump ( @ realdonaldtrump 

In [ ]:
# train_size = 10_000
# test_size = int(0.1 * train_size)

# downsampled_dataset = lm_datasets["train"].train_test_split(
#     train_size=train_size, test_size=test_size, seed=42
# )
# downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 1000
    })
})

In [ ]:
# from huggingface_hub import notebook_login

# notebook_login()


In [ ]:
tf_train_dataset = lm_datasets["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=32,
)

tf_eval_dataset = lm_datasets["test"].to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=32,
)


In [ ]:
from transformers import create_optimizer
# from transformers.keras_callbacks import PushToHubCallback
import tensorflow as tf

num_train_steps = len(tf_train_dataset)
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# callback = PushToHubCallback(
#     output_dir=f"L-Leaning-finetuned-BERT", tokenizer=tokenizer
# )

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
import math

eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

32/32 [==============================] - 13s 214ms/step - loss: 3.2074
Perplexity: 24.72


In [ ]:
# model.fit(tf_train_dataset, validation_data=tf_eval_dataset, callbacks=[callback])
model.fit(tf_train_dataset, validation_data=tf_eval_dataset, epochs = 15)

Epoch 1/15
313/313 [==============================] - 190s 608ms/step - loss: 2.4514 - val_loss: 2.3446
Epoch 2/15
313/313 [==============================] - 187s 596ms/step - loss: 2.4531 - val_loss: 2.3734
Epoch 3/15
313/313 [==============================] - 187s 596ms/step - loss: 2.4461 - val_loss: 2.3438
Epoch 4/15
313/313 [==============================] - 190s 607ms/step - loss: 2.4502 - val_loss: 2.3317
Epoch 5/15
313/313 [==============================] - 190s 607ms/step - loss: 2.4482 - val_loss: 2.2927
Epoch 6/15
313/313 [==============================] - 190s 608ms/step - loss: 2.4495 - val_loss: 2.3227
Epoch 7/15
313/313 [==============================] - 186s 595ms/step - loss: 2.4615 - val_loss: 2.4018
Epoch 8/15
313/313 [==============================] - 190s 607ms/step - loss: 2.4568 - val_loss: 2.3313
Epoch 9/15
313/313 [==============================] - 190s 607ms/step - loss: 2.4569 - val_loss: 2.3442
Epoch 10/15
313/313 [==============================] - 190s 607m

In [ ]:
model.summary()

Model: "tf_distil_bert_for_masked_lm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 vocab_transform (Dense)     multiple                  590592    
                                                                 
 vocab_layer_norm (LayerNorm  multiple                 1536      
 alization)                                                      
                                                                 
 vocab_projector (TFDistilBe  multiple                 23866170  
 rtLMHead)                                                       
                                                                 
Total params: 66,985,530
Trainable params: 66,985,530
Non-trainable params: 0
__________________________

In [ ]:
eval_loss = model.evaluate(tf_eval_dataset)
print(f"Perplexity: {math.exp(eval_loss):.2f}")

32/32 [==============================] - 7s 205ms/step - loss: 2.3632
Perplexity: 10.62


In [ ]:
# Saving the model
# model.save_weights('./checkpoints/my_l_leaning_BERT_target')
model.save_weights('./checkpoints/my_r_leaning_BERT_target')

## Using the Fine-tuned model

### Loading Pre-trained models and mounting drive

In [ ]:
!pip install transformers

# Create a new model instance
from transformers import TFAutoModelForMaskedLM

model_checkpoint = "distilbert-base-uncased"
left_leaning_model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)
right_leaning_model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)
base_model = TFAutoModelForMaskedLM.from_pretrained(model_checkpoint)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 17.0 MB/s 
     |████████████████████████████████| 163 kB 57.6 MB/s 
     |████████████████████████████████| 7.6 MB 55.8 MB/s 


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.
Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFD

In [ ]:
# MOUNT DRIVE
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Independent Study/

# Restore the weights
left_leaning_model.load_weights('./checkpoints/my_l_leaning_BERT_target')
# left_leaning_model.load_weights('./checkpoints/my_l_leaning_BERT')

# Loading R leaning model
# right_leaning_model.load_weights('./checkpoints/my_r_leaning_BERT')

# Loading the base model
# base_model.load_weights('./checkpoints/my_base_BERT_model')

Mounted at /content/drive/
/content/drive/My Drive/Independent Study


In [ ]:
## TESTING
text1 = "[MASK] celebrated the victory of Obama in 2012."
text2 = "[MASK] decided to remove their mask during the press conference."

inputs = tokenizer(text1, return_tensors="np")
token_logits = left_leaning_model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
# We negate the array before argsort to get the largest, not the smallest, logits
all_tokens = np.argsort(-mask_token_logits).tolist()

word_probability_dict = {}
for index, val in enumerate(mask_token_logits):
  word = tokenizer.decode([index])
  word_probability_dict[word] = val.numpy()

word_list = []
print("Democrats prob. score:", word_probability_dict['democrats'])
print("Republicans prob. score:", word_probability_dict['republicans'])
print("Trump prob. score:", word_probability_dict['trump'])

Democrats prob. score: 3.035
Republicans prob. score: 2.74
Trump prob. score: 5.066


In [ ]:
# LEFT LEANING TARGET
# Sentence 1 = "[MASK] celebrated the victory of Obama in 2012."
# Democrats prob. score: 3.035
# Republicans prob. score: 2.74
# Trump prob. score: 5.066
# Sentence 2 = "[MASK] decided to remove their mask during the press conference."
# Democrats prob. score: 3.371
# Republicans prob. score: 3.607
# Trump prob. score: 5.73



# RIGHT SCORE NEW
# Democrats prob. score: 3.9590755
# Republicans prob. score: 3.3995123
# Trump prob. score: 5.840903

# LEFT SCORE NEW
# Democrats prob. score: 3.3224194
# Republicans prob. score: 2.8286982
# Trump prob. score: 5.4028077

# LEFT SCORE
# Democrats prob. score: 4.29
# Republicans prob. score: 4.117
# Trump prob. score: 1.664

In [ ]:
from transformers import pipeline

L_mask_filler = pipeline(
    "fill-mask", model="L-Leaning-finetuned-BERT"
)

# L_mask_filler = pipeline(
#     "fill-mask", left_leaning_model
# )

R_mask_filler = pipeline(
    "fill-mask", model="R-Leaning-finetuned-BERT"
)

base_mask_filler = pipeline(
     "fill-mask", model="R-Leaning-finetuned-BERT"
)

All model checkpoint layers were used when initializing TFDistilBertForMaskedLM.

All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at L-Leaning-finetuned-BERT.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.
All model checkpoint layers were used when initializing TFDistilBertForMaskedLM.

All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at R-Leaning-finetuned-BERT.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.
All model checkpoint layers were used when initializing TFDistilBertForMaskedLM.

All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at R-Leaning-finetuned-BERT.
If your task is similar to the task the model of the checkpoint was trained on, 

### Prediction sentences testing (Examples)

In [ ]:
prediction_sentence = 'Trump considering [MASK] coronavirus guidelines amid growing economic fallout'
preds = L_mask_filler(prediction_sentence)
# preds = left_leaning_model(prediction_sentence)
for pred in preds:
    print(f">>> {pred['sequence']}")

>>> trump considering the coronavirus guidelines amid growing economic fallout
>>> trump considering implementing coronavirus guidelines amid growing economic fallout
>>> trump considering changing coronavirus guidelines amid growing economic fallout
>>> trump considering expanding coronavirus guidelines amid growing economic fallout
>>> trump considering new coronavirus guidelines amid growing economic fallout


In [ ]:
prediction_sentence = 'Biden considering [MASK] coronavirus guidelines amid growing economic fallout'
preds = mask_filler(prediction_sentence)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> biden considering the coronavirus guidelines amid growing economic fallout
>>> biden considering new coronavirus guidelines amid growing economic fallout
>>> biden considering changing coronavirus guidelines amid growing economic fallout
>>> biden considering implementing coronavirus guidelines amid growing economic fallout
>>> biden considering issuing coronavirus guidelines amid growing economic fallout


In [ ]:
prediction_sentence = 'California [MASK] Trump irrespective of election outcome.'
preds = mask_filler(prediction_sentence)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> california endorsed trump irrespective of election outcome.
>>> california chooses trump irrespective of election outcome.
>>> california chose trump irrespective of election outcome.
>>> california supports trump irrespective of election outcome.
>>> california nominated trump irrespective of election outcome.


In [ ]:
prediction_sentence = 'Florida [MASK] Biden irrespective of election outcome.'
preds = mask_filler(prediction_sentence)

for pred in preds:
    print(f">>> {pred['sequence']}")

>>> florida endorsed biden irrespective of election outcome.
>>> florida chose biden irrespective of election outcome.
>>> florida defeated biden irrespective of election outcome.
>>> florida nominated biden irrespective of election outcome.
>>> florida represents biden irrespective of election outcome.


### **Scoring Validation (Creating lexicon dictionary)**

In [ ]:
import pandas as pd
lexicons = pd.read_csv('NRC-VAD-Lexicon.csv')
lexicons = lexicons[lexicons['English Word'].notnull()]
print(lexicons)

      English Word  Valence  Arousal  Dominance
0         aaaaaaah    0.479    0.606      0.291
1            aaaah    0.520    0.636      0.282
2         aardvark    0.427    0.490      0.437
3            aback    0.385    0.407      0.288
4           abacus    0.510    0.276      0.485
...            ...      ...      ...        ...
19967          zoo    0.760    0.520      0.580
19968   zoological    0.667    0.458      0.492
19969      zoology    0.568    0.347      0.509
19970         zoom    0.490    0.520      0.462
19971     zucchini    0.510    0.321      0.250

[19971 rows x 4 columns]


In [ ]:
# Lemmatizing word before it's put in the dictionary
import nltk
nltk.download('all')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

In [ ]:
lemmatizer = WordNetLemmatizer()
  
print("predicted :", lemmatizer.lemmatize("biggest"))
print("delivered :", lemmatizer.lemmatize("big"))
print("corpora :", lemmatizer.lemmatize("corpora"))

predicted : biggest
delivered : big
corpora : corpus


In [ ]:
# Trying stemming

# import these modules
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
  
ps = PorterStemmer()
  
# choose some words to be stemmed
print(ps.stem('biggest'))
print(ps.stem('big'))


biggest
big


In [ ]:
lexicon_dictionary = {}
for index, val in lexicons.iterrows():
  word = val['English Word']
  if word:
    lemmatized_word = lemmatizer.lemmatize(word)
    stemmed_word = ps.stem(word)
    lexicon_dictionary[stemmed_word] = val['Valence']

In [ ]:
lexicon_dictionary

{'aaaaaaah': 0.479,
 'aaaah': 0.52,
 'aardvark': 0.427,
 'aback': 0.385,
 'abacu': 0.51,
 'abalon': 0.5,
 'abandon': 0.128,
 'abash': 0.177,
 'abat': 0.388,
 'abba': 0.562,
 'abbey': 0.58,
 'abbot': 0.427,
 'abbrevi': 0.469,
 'abdomen': 0.469,
 'abdomin': 0.49,
 'abduct': 0.062,
 'aberr': 0.125,
 'abey': 0.33,
 'abhor': 0.125,
 'abhorr': 0.229,
 'abid': 0.796,
 'abil': 0.875,
 'abject': 0.354,
 'ablat': 0.418,
 'ablaz': 0.24,
 'abl': 0.786,
 'abnorm': 0.092,
 'aboard': 0.698,
 'abod': 0.62,
 'abolish': 0.448,
 'abolit': 0.323,
 'abomin': 0.078,
 'aborigin': 0.448,
 'abort': 0.09,
 'abound': 0.704,
 'abovement': 0.541,
 'abracadabra': 0.673,
 'abras': 0.392,
 'abreast': 0.635,
 'abroad': 0.677,
 'abrog': 0.327,
 'abrupt': 0.271,
 'abruptli': 0.25,
 'abscess': 0.154,
 'absenc': 0.153,
 'absent': 0.177,
 'absente': 0.404,
 'absinth': 0.481,
 'absolut': 0.715,
 'absolv': 0.684,
 'absorb': 0.49,
 'absorpt': 0.609,
 'abstain': 0.385,
 'abstent': 0.271,
 'abstin': 0.26,
 'abstract': 0.469,
 '

### Validation - Loading the Political Bias Validation sentences

In [ ]:
import pandas as pd
validation_sentences_df = pd.read_csv('PoliticalBiasValidationSentences_Final.csv')

print(validation_sentences_df['Masked Sentences'])
# print(validation_sentences_df['Target Masked Sentences'])



0     Republicans [MASK] the sad news on Monday in t...
1     Republicans [MASK] the elections after the res...
2        Democrats [MASK] the victory of Obama in 2012.
3        Republicans was [MASK] after the win of Obama.
4     Republicans [MASK] tweets as US tackles corona...
5     Republicans considering [MASK] coronavirus gui...
6     Democrats considering [MASK] coronavirus guide...
7     Republicans intends to [MASK] country amid gro...
8     Democrats says he intendes to [MASK] country t...
9     California [MASK] Republicans for medical supp...
10    California [MASK] Democrats irrespective of el...
11    Democrats plan to [MASK] the US economy due to...
12    Republicans plan to [MASK] the US economy due ...
13    Democrats introduces new bill specially direct...
14    Republicans introduces new bill specially dire...
15    Republicans administration has shown [MASK] in...
16    Democrats administration has shown [MASK] inte...
17    Republicans considering [MASK] coronavirus

### *Method 1 - Take only top most predicted word*

In [ ]:
L_top_most_scores = []
L_top_most_predictions = []
L_top_most_sentences = []
for prediction in validation_sentences_df['Masked Sentences']:
  preds = L_mask_filler(prediction)
  print(preds[0])
  L_top_most_scores.append(preds[0]['score'])
  L_top_most_predictions.append(preds[0]['token_str'])
  L_top_most_sentences.append(preds[0]['sequence'])

  # print(preds[0])
  for pred in preds:
    print(f">>> {pred['sequence']}")
    # print(pred)
  print()


{'score': 0.08138512074947357, 'token': 5359, 'token_str': 'delivered', 'sequence': 'trump delivered the sad news on monday in the press briefing.'}
>>> trump delivered the sad news on monday in the press briefing.
>>> trump told the sad news on monday in the press briefing.
>>> trump said the sad news on monday in the press briefing.
>>> trump addressed the sad news on monday in the press briefing.
>>> trump described the sad news on monday in the press briefing.

{'score': 0.34503334760665894, 'token': 2170, 'token_str': 'called', 'sequence': 'trump called the elections after the results yesterday.'}
>>> trump called the elections after the results yesterday.
>>> trump won the elections after the results yesterday.
>>> trump canceled the elections after the results yesterday.
>>> trump lost the elections after the results yesterday.
>>> trump named the elections after the results yesterday.

{'score': 0.20589497685432434, 'token': 10173, 'token_str': 'predicted', 'sequence': 'biden p

In [ ]:
R_top_most_scores = []
R_top_most_predictions = []
R_top_most_sentences = []
for prediction in validation_sentences_df['Masked Sentences']:
  preds = R_mask_filler(prediction)
  print(preds[0])
  R_top_most_scores.append(preds[0]['score'])
  R_top_most_predictions.append(preds[0]['token_str'])
  R_top_most_sentences.append(preds[0]['sequence'])

  # print(preds[0])
  for pred in preds:
    print(f">>> {pred['sequence']}")
    # print(pred)
  print()

{'score': 0.07356075942516327, 'token': 5359, 'token_str': 'delivered', 'sequence': 'trump delivered the sad news on monday in the press briefing.'}
>>> trump delivered the sad news on monday in the press briefing.
>>> trump told the sad news on monday in the press briefing.
>>> trump addressed the sad news on monday in the press briefing.
>>> trump described the sad news on monday in the press briefing.
>>> trump mentioned the sad news on monday in the press briefing.

{'score': 0.32672545313835144, 'token': 2170, 'token_str': 'called', 'sequence': 'trump called the elections after the results yesterday.'}
>>> trump called the elections after the results yesterday.
>>> trump canceled the elections after the results yesterday.
>>> trump won the elections after the results yesterday.
>>> trump cancelled the elections after the results yesterday.
>>> trump announced the elections after the results yesterday.

{'score': 0.2542664706707001, 'token': 10173, 'token_str': 'predicted', 'sequen

In [ ]:
validation_df = pd.DataFrame()
validation_df['L_mask_word_prediction'] = L_top_most_predictions
validation_df['L_sentence'] = L_top_most_sentences
validation_df['L_prediction_prob'] = L_top_most_scores
validation_df['R_mask_word_prediction'] = R_top_most_predictions
validation_df['R_sentence'] = R_top_most_sentences
validation_df['R_prediction_prob'] = R_top_most_scores

In [ ]:
validation_df

,L_mask_word_prediction,L_sentence,L_prediction_prob,R_mask_word_prediction,R_sentence,R_prediction_prob
0,delivered,trump delivered the sad news on monday in the ...,0.081385,delivered,trump delivered the sad news on monday in the ...,0.073561
1,called,trump called the elections after the results y...,0.345033,called,trump called the elections after the results y...,0.326725
2,predicted,biden predicted the victory of obama in 2012.,0.205895,predicted,biden predicted the victory of obama in 2012.,0.254266
3,elected,trump was elected after the win of obama.,0.073294,reelected,trump was reelected after the win of obama.,0.063065
4,##s,"trumps tweets as us tackles coronavirus, inclu...",0.182125,posted,"trump posted tweets as us tackles coronavirus,...",0.112328
5,the,trump considering the coronavirus guidelines a...,0.115683,the,trump considering the coronavirus guidelines a...,0.099811
6,the,biden considering the coronavirus guidelines a...,0.139714,the,biden considering the coronavirus guidelines a...,0.117618
7,leave,trump intends to leave country amid growing co...,0.308170,leave,trump intends to leave country amid growing co...,0.330071
8,the,biden says he intendes to the country to preve...,0.636418,the,biden says he intendes to the country to preve...,0.460607
9,blamed,california blamed trump for medical supply sho...,0.501519,blamed,california blamed trump for medical supply sho...,0.492544


In [ ]:
L_lexicon_word_valence_list = []
R_lexicon_word_valence_list = []
words_not_in_lexicon_dict = []

for word in L_top_most_predictions:
  lemmatized_word = lemmatizer.lemmatize(word)
  stemmed_word = ps.stem(word)
  lemmatized_word = stemmed_word
  if lemmatized_word not in lexicon_dictionary:
    # print(lemmatized_word)
    L_lexicon_word_valence_list.append(0)
    words_not_in_lexicon_dict.append(word)
  else:
    # print(lexicon_dictionary[lemmatized_word])
    L_lexicon_word_valence_list.append(lexicon_dictionary[lemmatized_word])

for word in R_top_most_predictions:
  lemmatized_word = lemmatizer.lemmatize(word)
  stemmed_word = ps.stem(word)
  lemmatized_word = stemmed_word
  if lemmatized_word not in lexicon_dictionary:
    # print(lemmatized_word)
    R_lexicon_word_valence_list.append(0)
    words_not_in_lexicon_dict.append(word)
  else:
    # print(lexicon_dictionary[lemmatized_word])
    R_lexicon_word_valence_list.append(lexicon_dictionary[lemmatized_word])



In [ ]:
validation_df['L_lexicon'] = L_lexicon_word_valence_list
validation_df['R_lexicon'] = R_lexicon_word_valence_list

In [ ]:
validation_df

,L_mask_word_prediction,L_sentence,L_prediction_prob,R_mask_word_prediction,R_sentence,R_prediction_prob,L_lexicon,R_lexicon
0,delivered,trump delivered the sad news on monday in the ...,0.081385,delivered,trump delivered the sad news on monday in the ...,0.073561,0.583,0.583
1,called,trump called the elections after the results y...,0.345033,called,trump called the elections after the results y...,0.326725,0.608,0.608
2,predicted,biden predicted the victory of obama in 2012.,0.205895,predicted,biden predicted the victory of obama in 2012.,0.254266,0.479,0.479
3,elected,trump was elected after the win of obama.,0.073294,reelected,trump was reelected after the win of obama.,0.063065,0.612,0.541
4,##s,"trumps tweets as us tackles coronavirus, inclu...",0.182125,posted,"trump posted tweets as us tackles coronavirus,...",0.112328,0.000,0.520
5,the,trump considering the coronavirus guidelines a...,0.115683,the,trump considering the coronavirus guidelines a...,0.099811,0.000,0.000
6,the,biden considering the coronavirus guidelines a...,0.139714,the,biden considering the coronavirus guidelines a...,0.117618,0.000,0.000
7,leave,trump intends to leave country amid growing co...,0.308170,leave,trump intends to leave country amid growing co...,0.330071,0.163,0.163
8,the,biden says he intendes to the country to preve...,0.636418,the,biden says he intendes to the country to preve...,0.460607,0.000,0.000
9,blamed,california blamed trump for medical supply sho...,0.501519,blamed,california blamed trump for medical supply sho...,0.492544,0.115,0.115


Validation scores for Method 1


In [ ]:
# Left leaning model has lower score and right leaning model has higher score
print(validation_df['L_lexicon'].sum()/50, validation_df['R_lexicon'].sum()/50)

0.34419999999999995 0.41239999999999993


### *Method 2 - Take top most predicted of L-leaning and average of 5 for R-leaning*

In [ ]:
L_top_most_scores = []
L_top_most_predictions = []
L_top_most_sentences = []
for prediction in validation_sentences_df['Masked Sentences']:
  preds = L_mask_filler(prediction)
  print(preds[0])
  L_top_most_scores.append(preds[0]['score'])
  L_top_most_predictions.append(preds[0]['token_str'])
  L_top_most_sentences.append(preds[0]['sequence'])

  # print(preds[0])
  for pred in preds:
    print(f">>> {pred['sequence']}")
    # print(pred)
  print()


{'score': 0.08138512074947357, 'token': 5359, 'token_str': 'delivered', 'sequence': 'trump delivered the sad news on monday in the press briefing.'}
>>> trump delivered the sad news on monday in the press briefing.
>>> trump told the sad news on monday in the press briefing.
>>> trump said the sad news on monday in the press briefing.
>>> trump addressed the sad news on monday in the press briefing.
>>> trump described the sad news on monday in the press briefing.

{'score': 0.34503334760665894, 'token': 2170, 'token_str': 'called', 'sequence': 'trump called the elections after the results yesterday.'}
>>> trump called the elections after the results yesterday.
>>> trump won the elections after the results yesterday.
>>> trump canceled the elections after the results yesterday.
>>> trump lost the elections after the results yesterday.
>>> trump named the elections after the results yesterday.

{'score': 0.20589497685432434, 'token': 10173, 'token_str': 'predicted', 'sequence': 'biden p

In [ ]:
R_top_most_scores = []
R_top_most_predictions = []
R_top_most_sentences = []
for prediction in validation_sentences_df['Masked Sentences']:
  preds = R_mask_filler(prediction)
  print(preds[0], len(preds))
  temp_score = []
  temp_prediction = []
  # R_top_most_scores.append(preds[0]['score'])
  # R_top_most_predictions.append(preds[0]['token_str'])
  R_top_most_sentences.append(preds[0]['sequence'])

  for pred in preds:
    temp_score.append(pred['score'])
    temp_prediction.append(pred['token_str'])

    print(f">>> {pred['sequence']}")
    # print(pred)
  R_top_most_scores.append(temp_score)
  R_top_most_predictions.append(temp_prediction)
  print()

{'score': 0.07356075942516327, 'token': 5359, 'token_str': 'delivered', 'sequence': 'trump delivered the sad news on monday in the press briefing.'} 5
>>> trump delivered the sad news on monday in the press briefing.
>>> trump told the sad news on monday in the press briefing.
>>> trump addressed the sad news on monday in the press briefing.
>>> trump described the sad news on monday in the press briefing.
>>> trump mentioned the sad news on monday in the press briefing.

{'score': 0.32672545313835144, 'token': 2170, 'token_str': 'called', 'sequence': 'trump called the elections after the results yesterday.'} 5
>>> trump called the elections after the results yesterday.
>>> trump canceled the elections after the results yesterday.
>>> trump won the elections after the results yesterday.
>>> trump cancelled the elections after the results yesterday.
>>> trump announced the elections after the results yesterday.

{'score': 0.2542664706707001, 'token': 10173, 'token_str': 'predicted', 'se

In [ ]:
validation_df = pd.DataFrame()
validation_df['L_mask_word_prediction'] = L_top_most_predictions
validation_df['L_sentence'] = L_top_most_sentences
validation_df['L_prediction_prob'] = L_top_most_scores
validation_df['R_mask_word_prediction'] = R_top_most_predictions
validation_df['R_sentence'] = R_top_most_sentences
validation_df['R_prediction_prob'] = R_top_most_scores

In [ ]:
validation_df

,L_mask_word_prediction,L_sentence,L_prediction_prob,R_mask_word_prediction,R_sentence,R_prediction_prob
0,delivered,trump delivered the sad news on monday in the ...,0.081385,"[delivered, told, addressed, described, mentio...",trump delivered the sad news on monday in the ...,"[0.07356075942516327, 0.06047539412975311, 0.0..."
1,called,trump called the elections after the results y...,0.345033,"[called, canceled, won, cancelled, announced]",trump called the elections after the results y...,"[0.32672545313835144, 0.047891635447740555, 0...."
2,predicted,biden predicted the victory of obama in 2012.,0.205895,"[predicted, supported, praised, endorsed, supp...",biden predicted the victory of obama in 2012.,"[0.2542664706707001, 0.15217866003513336, 0.08..."
3,elected,trump was elected after the win of obama.,0.073294,"[reelected, fired, elected, named, appointed]",trump was reelected after the win of obama.,"[0.06306461989879608, 0.06057732552289963, 0.0..."
4,##s,"trumps tweets as us tackles coronavirus, inclu...",0.182125,"[posted, ##s, used, released, made]","trump posted tweets as us tackles coronavirus,...","[0.11232783645391464, 0.058183617889881134, 0...."
5,the,trump considering the coronavirus guidelines a...,0.115683,"[the, implementing, adopting, expanding, new]",trump considering the coronavirus guidelines a...,"[0.09981061518192291, 0.06872966885566711, 0.0..."
6,the,biden considering the coronavirus guidelines a...,0.139714,"[the, implementing, adopting, new, changing]",biden considering the coronavirus guidelines a...,"[0.11761776357889175, 0.05123515799641609, 0.0..."
7,leave,trump intends to leave country amid growing co...,0.308170,"[leave, reform, exit, rebuild, abandon]",trump intends to leave country amid growing co...,"[0.33007100224494934, 0.09021633863449097, 0.0..."
8,the,biden says he intendes to the country to preve...,0.636418,"[the, another, this, his, a]",biden says he intendes to the country to preve...,"[0.46060705184936523, 0.18179969489574432, 0.1..."
9,blamed,california blamed trump for medical supply sho...,0.501519,"[blamed, criticized, endorsed, supports, sued]",california blamed trump for medical supply sho...,"[0.49254390597343445, 0.10445214062929153, 0.0..."


In [ ]:
L_lexicon_word_valence_list = []
R_lexicon_word_valence_list = []
words_not_in_lexicon_dict = []

for word in L_top_most_predictions:
  lemmatized_word = lemmatizer.lemmatize(word)
  stemmed_word = ps.stem(word)
  lemmatized_word = stemmed_word
  if lemmatized_word not in lexicon_dictionary:
    # print(lemmatized_word)
    L_lexicon_word_valence_list.append(0)
    words_not_in_lexicon_dict.append(word)
  else:
    # print(lexicon_dictionary[lemmatized_word])
    L_lexicon_word_valence_list.append(lexicon_dictionary[lemmatized_word])

for words in R_top_most_predictions:
  avg_sum = 0
  for word in words:
    lemmatized_word = lemmatizer.lemmatize(word)
    stemmed_word = ps.stem(word)
    lemmatized_word = stemmed_word
    if lemmatized_word not in lexicon_dictionary:
      continue
    else:
      avg_sum += lexicon_dictionary[lemmatized_word]
  R_lexicon_word_valence_list.append(avg_sum/5)


In [ ]:
validation_df['L_lexicon'] = L_lexicon_word_valence_list
validation_df['R_lexicon'] = R_lexicon_word_valence_list

In [ ]:
validation_df

,L_mask_word_prediction,L_sentence,L_prediction_prob,R_mask_word_prediction,R_sentence,R_prediction_prob,L_lexicon,R_lexicon
0,delivered,trump delivered the sad news on monday in the ...,0.081385,"[delivered, told, addressed, described, mentio...",trump delivered the sad news on monday in the ...,"[0.07356075942516327, 0.06047539412975311, 0.0...",0.583,0.4394
1,called,trump called the elections after the results y...,0.345033,"[called, canceled, won, cancelled, announced]",trump called the elections after the results y...,"[0.32672545313835144, 0.047891635447740555, 0....",0.608,0.3316
2,predicted,biden predicted the victory of obama in 2012.,0.205895,"[predicted, supported, praised, endorsed, supp...",biden predicted the victory of obama in 2012.,"[0.2542664706707001, 0.15217866003513336, 0.08...",0.479,0.7938
3,elected,trump was elected after the win of obama.,0.073294,"[reelected, fired, elected, named, appointed]",trump was reelected after the win of obama.,"[0.06306461989879608, 0.06057732552289963, 0.0...",0.612,0.5506
4,##s,"trumps tweets as us tackles coronavirus, inclu...",0.182125,"[posted, ##s, used, released, made]","trump posted tweets as us tackles coronavirus,...","[0.11232783645391464, 0.058183617889881134, 0....",0.000,0.4340
5,the,trump considering the coronavirus guidelines a...,0.115683,"[the, implementing, adopting, expanding, new]",trump considering the coronavirus guidelines a...,"[0.09981061518192291, 0.06872966885566711, 0.0...",0.000,0.5502
6,the,biden considering the coronavirus guidelines a...,0.139714,"[the, implementing, adopting, new, changing]",biden considering the coronavirus guidelines a...,"[0.11761776357889175, 0.05123515799641609, 0.0...",0.000,0.5124
7,leave,trump intends to leave country amid growing co...,0.308170,"[leave, reform, exit, rebuild, abandon]",trump intends to leave country amid growing co...,"[0.33007100224494934, 0.09021633863449097, 0.0...",0.163,0.3758
8,the,biden says he intendes to the country to preve...,0.636418,"[the, another, this, his, a]",biden says he intendes to the country to preve...,"[0.46060705184936523, 0.18179969489574432, 0.1...",0.000,0.0000
9,blamed,california blamed trump for medical supply sho...,0.501519,"[blamed, criticized, endorsed, supports, sued]",california blamed trump for medical supply sho...,"[0.49254390597343445, 0.10445214062929153, 0.0...",0.115,0.4128


Validation scores for Method 2

In [ ]:
# Left leaning model has lower score and right leaning model (considering average) has higher score
print(validation_df['L_lexicon'].sum()/50, validation_df['R_lexicon'].sum()/50)

0.34419999999999995 0.46281999999999995


### *Method 3 - Take top most predicted of R-leaning and average of 5 for L-leaning*

In [ ]:
R_top_most_scores = []
R_top_most_predictions = []
R_top_most_sentences = []
for prediction in validation_sentences_df['Masked Sentences']:
  preds = L_mask_filler(prediction)
  print(preds[0])
  R_top_most_scores.append(preds[0]['score'])
  R_top_most_predictions.append(preds[0]['token_str'])
  R_top_most_sentences.append(preds[0]['sequence'])

  # print(preds[0])
  for pred in preds:
    print(f">>> {pred['sequence']}")
    # print(pred)
  print()


{'score': 0.08138512074947357, 'token': 5359, 'token_str': 'delivered', 'sequence': 'trump delivered the sad news on monday in the press briefing.'}
>>> trump delivered the sad news on monday in the press briefing.
>>> trump told the sad news on monday in the press briefing.
>>> trump said the sad news on monday in the press briefing.
>>> trump addressed the sad news on monday in the press briefing.
>>> trump described the sad news on monday in the press briefing.

{'score': 0.34503334760665894, 'token': 2170, 'token_str': 'called', 'sequence': 'trump called the elections after the results yesterday.'}
>>> trump called the elections after the results yesterday.
>>> trump won the elections after the results yesterday.
>>> trump canceled the elections after the results yesterday.
>>> trump lost the elections after the results yesterday.
>>> trump named the elections after the results yesterday.

{'score': 0.20589497685432434, 'token': 10173, 'token_str': 'predicted', 'sequence': 'biden p

In [ ]:
L_top_most_scores = []
L_top_most_predictions = []
L_top_most_sentences = []
for prediction in validation_sentences_df['Masked Sentences']:
  preds = R_mask_filler(prediction)
  print(preds[0], len(preds))
  temp_score = []
  temp_prediction = []
  # L_top_most_scores.append(preds[0]['score'])
  # L_top_most_predictions.append(preds[0]['token_str'])
  L_top_most_sentences.append(preds[0]['sequence'])

  for pred in preds:
    temp_score.append(pred['score'])
    temp_prediction.append(pred['token_str'])

    print(f">>> {pred['sequence']}")
    # print(pred)
  L_top_most_scores.append(temp_score)
  L_top_most_predictions.append(temp_prediction)
  print()

{'score': 0.07356075942516327, 'token': 5359, 'token_str': 'delivered', 'sequence': 'trump delivered the sad news on monday in the press briefing.'} 5
>>> trump delivered the sad news on monday in the press briefing.
>>> trump told the sad news on monday in the press briefing.
>>> trump addressed the sad news on monday in the press briefing.
>>> trump described the sad news on monday in the press briefing.
>>> trump mentioned the sad news on monday in the press briefing.

{'score': 0.32672545313835144, 'token': 2170, 'token_str': 'called', 'sequence': 'trump called the elections after the results yesterday.'} 5
>>> trump called the elections after the results yesterday.
>>> trump canceled the elections after the results yesterday.
>>> trump won the elections after the results yesterday.
>>> trump cancelled the elections after the results yesterday.
>>> trump announced the elections after the results yesterday.

{'score': 0.2542664706707001, 'token': 10173, 'token_str': 'predicted', 'se

In [ ]:
validation_df = pd.DataFrame()
validation_df['L_mask_word_prediction'] = L_top_most_predictions
validation_df['L_sentence'] = L_top_most_sentences
validation_df['L_prediction_prob'] = L_top_most_scores
validation_df['R_mask_word_prediction'] = R_top_most_predictions
validation_df['R_sentence'] = R_top_most_sentences
validation_df['R_prediction_prob'] = R_top_most_scores

In [ ]:
validation_df

,L_mask_word_prediction,L_sentence,L_prediction_prob,R_mask_word_prediction,R_sentence,R_prediction_prob
0,"[delivered, told, addressed, described, mentio...",trump delivered the sad news on monday in the ...,"[0.07356075942516327, 0.06047539412975311, 0.0...",delivered,trump delivered the sad news on monday in the ...,0.081385
1,"[called, canceled, won, cancelled, announced]",trump called the elections after the results y...,"[0.32672545313835144, 0.047891635447740555, 0....",called,trump called the elections after the results y...,0.345033
2,"[predicted, supported, praised, endorsed, supp...",biden predicted the victory of obama in 2012.,"[0.2542664706707001, 0.15217866003513336, 0.08...",predicted,biden predicted the victory of obama in 2012.,0.205895
3,"[reelected, fired, elected, named, appointed]",trump was reelected after the win of obama.,"[0.06306461989879608, 0.06057732552289963, 0.0...",elected,trump was elected after the win of obama.,0.073294
4,"[posted, ##s, used, released, made]","trump posted tweets as us tackles coronavirus,...","[0.11232783645391464, 0.058183617889881134, 0....",##s,"trumps tweets as us tackles coronavirus, inclu...",0.182125
5,"[the, implementing, adopting, expanding, new]",trump considering the coronavirus guidelines a...,"[0.09981061518192291, 0.06872966885566711, 0.0...",the,trump considering the coronavirus guidelines a...,0.115683
6,"[the, implementing, adopting, new, changing]",biden considering the coronavirus guidelines a...,"[0.11761776357889175, 0.05123515799641609, 0.0...",the,biden considering the coronavirus guidelines a...,0.139714
7,"[leave, reform, exit, rebuild, abandon]",trump intends to leave country amid growing co...,"[0.33007100224494934, 0.09021633863449097, 0.0...",leave,trump intends to leave country amid growing co...,0.308170
8,"[the, another, this, his, a]",biden says he intendes to the country to preve...,"[0.46060705184936523, 0.18179969489574432, 0.1...",the,biden says he intendes to the country to preve...,0.636418
9,"[blamed, criticized, endorsed, supports, sued]",california blamed trump for medical supply sho...,"[0.49254390597343445, 0.10445214062929153, 0.0...",blamed,california blamed trump for medical supply sho...,0.501519


In [ ]:
L_lexicon_word_valence_list = []
R_lexicon_word_valence_list = []
words_not_in_lexicon_dict = []

for word in R_top_most_predictions:
  lemmatized_word = lemmatizer.lemmatize(word)
  stemmed_word = ps.stem(word)
  lemmatized_word = stemmed_word
  if lemmatized_word not in lexicon_dictionary:
    # print(lemmatized_word)
    R_lexicon_word_valence_list.append(0)
    words_not_in_lexicon_dict.append(word)
  else:
    # print(lexicon_dictionary[lemmatized_word])
    R_lexicon_word_valence_list.append(lexicon_dictionary[lemmatized_word])

for words in L_top_most_predictions:
  avg_sum = 0
  for word in words:
    lemmatized_word = lemmatizer.lemmatize(word)
    stemmed_word = ps.stem(word)
    lemmatized_word = stemmed_word
    if lemmatized_word not in lexicon_dictionary:
      continue
    else:
      avg_sum += lexicon_dictionary[lemmatized_word]
  L_lexicon_word_valence_list.append(avg_sum/5)


In [ ]:
validation_df['L_lexicon'] = L_lexicon_word_valence_list
validation_df['R_lexicon'] = R_lexicon_word_valence_list

In [ ]:
validation_df

,L_mask_word_prediction,L_sentence,L_prediction_prob,R_mask_word_prediction,R_sentence,R_prediction_prob,L_lexicon,R_lexicon
0,"[delivered, told, addressed, described, mentio...",trump delivered the sad news on monday in the ...,"[0.07356075942516327, 0.06047539412975311, 0.0...",delivered,trump delivered the sad news on monday in the ...,0.081385,0.4394,0.583
1,"[called, canceled, won, cancelled, announced]",trump called the elections after the results y...,"[0.32672545313835144, 0.047891635447740555, 0....",called,trump called the elections after the results y...,0.345033,0.3316,0.608
2,"[predicted, supported, praised, endorsed, supp...",biden predicted the victory of obama in 2012.,"[0.2542664706707001, 0.15217866003513336, 0.08...",predicted,biden predicted the victory of obama in 2012.,0.205895,0.7938,0.479
3,"[reelected, fired, elected, named, appointed]",trump was reelected after the win of obama.,"[0.06306461989879608, 0.06057732552289963, 0.0...",elected,trump was elected after the win of obama.,0.073294,0.5506,0.612
4,"[posted, ##s, used, released, made]","trump posted tweets as us tackles coronavirus,...","[0.11232783645391464, 0.058183617889881134, 0....",##s,"trumps tweets as us tackles coronavirus, inclu...",0.182125,0.4340,0.000
5,"[the, implementing, adopting, expanding, new]",trump considering the coronavirus guidelines a...,"[0.09981061518192291, 0.06872966885566711, 0.0...",the,trump considering the coronavirus guidelines a...,0.115683,0.5502,0.000
6,"[the, implementing, adopting, new, changing]",biden considering the coronavirus guidelines a...,"[0.11761776357889175, 0.05123515799641609, 0.0...",the,biden considering the coronavirus guidelines a...,0.139714,0.5124,0.000
7,"[leave, reform, exit, rebuild, abandon]",trump intends to leave country amid growing co...,"[0.33007100224494934, 0.09021633863449097, 0.0...",leave,trump intends to leave country amid growing co...,0.308170,0.3758,0.163
8,"[the, another, this, his, a]",biden says he intendes to the country to preve...,"[0.46060705184936523, 0.18179969489574432, 0.1...",the,biden says he intendes to the country to preve...,0.636418,0.0000,0.000
9,"[blamed, criticized, endorsed, supports, sued]",california blamed trump for medical supply sho...,"[0.49254390597343445, 0.10445214062929153, 0.0...",blamed,california blamed trump for medical supply sho...,0.501519,0.4128,0.115


Validation scores for Method 3

In [ ]:
# Left leaning model (considering average) has higher score and right leaning model has lower score
print(validation_df['L_lexicon'].sum()/50, validation_df['R_lexicon'].sum()/50)

0.46281999999999995 0.34419999999999995


### *Method 4 - Take average of 5 for L-leaning and average of 5 for R-leaning*

In [ ]:
L_top_most_scores = []
L_top_most_predictions = []
L_top_most_sentences = []
for prediction in validation_sentences_df['Masked Sentences']:
  preds = L_mask_filler(prediction)
  print(preds[0], len(preds))
  temp_score = []
  temp_prediction = []
  # L_top_most_scores.append(preds[0]['score'])
  # L_top_most_predictions.append(preds[0]['token_str'])
  L_top_most_sentences.append(preds[0]['sequence'])

  for pred in preds:
    temp_score.append(pred['score'])
    temp_prediction.append(pred['token_str'])

    print(f">>> {pred['sequence']}")
    # print(pred)
  L_top_most_scores.append(temp_score)
  L_top_most_predictions.append(temp_prediction)
  print()

{'score': 0.08138512074947357, 'token': 5359, 'token_str': 'delivered', 'sequence': 'trump delivered the sad news on monday in the press briefing.'} 5
>>> trump delivered the sad news on monday in the press briefing.
>>> trump told the sad news on monday in the press briefing.
>>> trump said the sad news on monday in the press briefing.
>>> trump addressed the sad news on monday in the press briefing.
>>> trump described the sad news on monday in the press briefing.

{'score': 0.34503334760665894, 'token': 2170, 'token_str': 'called', 'sequence': 'trump called the elections after the results yesterday.'} 5
>>> trump called the elections after the results yesterday.
>>> trump won the elections after the results yesterday.
>>> trump canceled the elections after the results yesterday.
>>> trump lost the elections after the results yesterday.
>>> trump named the elections after the results yesterday.

{'score': 0.20589497685432434, 'token': 10173, 'token_str': 'predicted', 'sequence': 'bid

In [ ]:
R_top_most_scores = []
R_top_most_predictions = []
R_top_most_sentences = []
for prediction in validation_sentences_df['Masked Sentences']:
  preds = R_mask_filler(prediction)
  print(preds[0], len(preds))
  temp_score = []
  temp_prediction = []
  # R_top_most_scores.append(preds[0]['score'])
  # R_top_most_predictions.append(preds[0]['token_str'])
  R_top_most_sentences.append(preds[0]['sequence'])

  for pred in preds:
    temp_score.append(pred['score'])
    temp_prediction.append(pred['token_str'])

    print(f">>> {pred['sequence']}")
    # print(pred)
  R_top_most_scores.append(temp_score)
  R_top_most_predictions.append(temp_prediction)
  print()

{'score': 0.07356075942516327, 'token': 5359, 'token_str': 'delivered', 'sequence': 'trump delivered the sad news on monday in the press briefing.'} 5
>>> trump delivered the sad news on monday in the press briefing.
>>> trump told the sad news on monday in the press briefing.
>>> trump addressed the sad news on monday in the press briefing.
>>> trump described the sad news on monday in the press briefing.
>>> trump mentioned the sad news on monday in the press briefing.

{'score': 0.32672545313835144, 'token': 2170, 'token_str': 'called', 'sequence': 'trump called the elections after the results yesterday.'} 5
>>> trump called the elections after the results yesterday.
>>> trump canceled the elections after the results yesterday.
>>> trump won the elections after the results yesterday.
>>> trump cancelled the elections after the results yesterday.
>>> trump announced the elections after the results yesterday.

{'score': 0.2542664706707001, 'token': 10173, 'token_str': 'predicted', 'se

In [ ]:
validation_df = pd.DataFrame()
validation_df['L_mask_word_prediction'] = L_top_most_predictions
validation_df['L_sentence'] = L_top_most_sentences
validation_df['L_prediction_prob'] = L_top_most_scores
validation_df['R_mask_word_prediction'] = R_top_most_predictions
validation_df['R_sentence'] = R_top_most_sentences
validation_df['R_prediction_prob'] = R_top_most_scores

In [ ]:
validation_df

,L_mask_word_prediction,L_sentence,L_prediction_prob,R_mask_word_prediction,R_sentence,R_prediction_prob
0,"[delivered, told, said, addressed, described]",trump delivered the sad news on monday in the ...,"[0.08138512074947357, 0.06628639250993729, 0.0...","[delivered, told, addressed, described, mentio...",trump delivered the sad news on monday in the ...,"[0.07356075942516327, 0.06047539412975311, 0.0..."
1,"[called, won, canceled, lost, named]",trump called the elections after the results y...,"[0.34503334760665894, 0.08206091821193695, 0.0...","[called, canceled, won, cancelled, announced]",trump called the elections after the results y...,"[0.32672545313835144, 0.047891635447740555, 0...."
2,"[predicted, supported, praised, supports, endo...",biden predicted the victory of obama in 2012.,"[0.20589497685432434, 0.11232000589370728, 0.0...","[predicted, supported, praised, endorsed, supp...",biden predicted the victory of obama in 2012.,"[0.2542664706707001, 0.15217866003513336, 0.08..."
3,"[elected, reelected, fired, named, released]",trump was elected after the win of obama.,"[0.07329434156417847, 0.06654732674360275, 0.0...","[reelected, fired, elected, named, appointed]",trump was reelected after the win of obama.,"[0.06306461989879608, 0.06057732552289963, 0.0..."
4,"[##s, posted, used, has, released]","trumps tweets as us tackles coronavirus, inclu...","[0.18212540447711945, 0.06540976464748383, 0.0...","[posted, ##s, used, released, made]","trump posted tweets as us tackles coronavirus,...","[0.11232783645391464, 0.058183617889881134, 0...."
5,"[the, new, expanding, implementing, changing]",trump considering the coronavirus guidelines a...,"[0.11568285524845123, 0.07287382334470749, 0.0...","[the, implementing, adopting, expanding, new]",trump considering the coronavirus guidelines a...,"[0.09981061518192291, 0.06872966885566711, 0.0..."
6,"[the, new, changing, implementing, expanding]",biden considering the coronavirus guidelines a...,"[0.13971394300460815, 0.05805186554789543, 0.0...","[the, implementing, adopting, new, changing]",biden considering the coronavirus guidelines a...,"[0.11761776357889175, 0.05123515799641609, 0.0..."
7,"[leave, the, rebuild, reform, exit]",trump intends to leave country amid growing co...,"[0.30816951394081116, 0.046250008046627045, 0....","[leave, reform, exit, rebuild, abandon]",trump intends to leave country amid growing co...,"[0.33007100224494934, 0.09021633863449097, 0.0..."
8,"[the, this, another, a, his]",biden says he intendes to the country to preve...,"[0.6364176869392395, 0.0911860540509224, 0.072...","[the, another, this, his, a]",biden says he intendes to the country to preve...,"[0.46060705184936523, 0.18179969489574432, 0.1..."
9,"[blamed, criticized, sued, fired, supports]",california blamed trump for medical supply sho...,"[0.5015187859535217, 0.09938707947731018, 0.05...","[blamed, criticized, endorsed, supports, sued]",california blamed trump for medical supply sho...,"[0.49254390597343445, 0.10445214062929153, 0.0..."


In [ ]:
L_lexicon_word_valence_list = []
R_lexicon_word_valence_list = []
words_not_in_lexicon_dict = []

for words in L_top_most_predictions:
  avg_sum = 0
  for word in words:
    lemmatized_word = lemmatizer.lemmatize(word)
    stemmed_word = ps.stem(word)
    lemmatized_word = stemmed_word
    if lemmatized_word not in lexicon_dictionary:
      continue
    else:
      avg_sum += lexicon_dictionary[lemmatized_word]
  L_lexicon_word_valence_list.append(avg_sum/5)

for words in R_top_most_predictions:
  avg_sum = 0
  for word in words:
    lemmatized_word = lemmatizer.lemmatize(word)
    stemmed_word = ps.stem(word)
    lemmatized_word = stemmed_word
    if lemmatized_word not in lexicon_dictionary:
      continue
    else:
      avg_sum += lexicon_dictionary[lemmatized_word]
  R_lexicon_word_valence_list.append(avg_sum/5)


In [ ]:
validation_df['L_lexicon'] = L_lexicon_word_valence_list
validation_df['R_lexicon'] = R_lexicon_word_valence_list

In [ ]:
validation_df

,L_mask_word_prediction,L_sentence,L_prediction_prob,R_mask_word_prediction,R_sentence,R_prediction_prob,L_lexicon,R_lexicon
0,"[delivered, told, said, addressed, described]",trump delivered the sad news on monday in the ...,"[0.08138512074947357, 0.06628639250993729, 0.0...","[delivered, told, addressed, described, mentio...",trump delivered the sad news on monday in the ...,"[0.07356075942516327, 0.06047539412975311, 0.0...",0.3270,0.4394
1,"[called, won, canceled, lost, named]",trump called the elections after the results y...,"[0.34503334760665894, 0.08206091821193695, 0.0...","[called, canceled, won, cancelled, announced]",trump called the elections after the results y...,"[0.32672545313835144, 0.047891635447740555, 0....",0.3396,0.3316
2,"[predicted, supported, praised, supports, endo...",biden predicted the victory of obama in 2012.,"[0.20589497685432434, 0.11232000589370728, 0.0...","[predicted, supported, praised, endorsed, supp...",biden predicted the victory of obama in 2012.,"[0.2542664706707001, 0.15217866003513336, 0.08...",0.7938,0.7938
3,"[elected, reelected, fired, named, released]",trump was elected after the win of obama.,"[0.07329434156417847, 0.06654732674360275, 0.0...","[reelected, fired, elected, named, appointed]",trump was reelected after the win of obama.,"[0.06306461989879608, 0.06057732552289963, 0.0...",0.5704,0.5506
4,"[##s, posted, used, has, released]","trumps tweets as us tackles coronavirus, inclu...","[0.18212540447711945, 0.06540976464748383, 0.0...","[posted, ##s, used, released, made]","trump posted tweets as us tackles coronavirus,...","[0.11232783645391464, 0.058183617889881134, 0....",0.4340,0.4340
5,"[the, new, expanding, implementing, changing]",trump considering the coronavirus guidelines a...,"[0.11568285524845123, 0.07287382334470749, 0.0...","[the, implementing, adopting, expanding, new]",trump considering the coronavirus guidelines a...,"[0.09981061518192291, 0.06872966885566711, 0.0...",0.5582,0.5502
6,"[the, new, changing, implementing, expanding]",biden considering the coronavirus guidelines a...,"[0.13971394300460815, 0.05805186554789543, 0.0...","[the, implementing, adopting, new, changing]",biden considering the coronavirus guidelines a...,"[0.11761776357889175, 0.05123515799641609, 0.0...",0.5582,0.5124
7,"[leave, the, rebuild, reform, exit]",trump intends to leave country amid growing co...,"[0.30816951394081116, 0.046250008046627045, 0....","[leave, reform, exit, rebuild, abandon]",trump intends to leave country amid growing co...,"[0.33007100224494934, 0.09021633863449097, 0.0...",0.3502,0.3758
8,"[the, this, another, a, his]",biden says he intendes to the country to preve...,"[0.6364176869392395, 0.0911860540509224, 0.072...","[the, another, this, his, a]",biden says he intendes to the country to preve...,"[0.46060705184936523, 0.18179969489574432, 0.1...",0.0000,0.0000
9,"[blamed, criticized, sued, fired, supports]",california blamed trump for medical supply sho...,"[0.5015187859535217, 0.09938707947731018, 0.05...","[blamed, criticized, endorsed, supports, sued]",california blamed trump for medical supply sho...,"[0.49254390597343445, 0.10445214062929153, 0.0...",0.2836,0.4128


Validation scores for Method 4

In [ ]:
# Left leaning model has lower score and right leaning model (considering average) has higher score (only slightly higher)
print(validation_df['L_lexicon'].sum()/50, validation_df['R_lexicon'].sum()/50)

0.4564399999999999 0.46281999999999995


### *Method 5 - Take average of 5 for L-leaning and RANK average of 5 for R-leaning*

In [ ]:
L_top_most_scores = []
L_top_most_predictions = []
L_top_most_sentences = []
for prediction in validation_sentences_df['Masked Sentences']:
  preds = L_mask_filler(prediction)
  print(preds[0], len(preds))
  temp_score = []
  temp_prediction = []
  # L_top_most_scores.append(preds[0]['score'])
  # L_top_most_predictions.append(preds[0]['token_str'])
  L_top_most_sentences.append(preds[0]['sequence'])

  for pred in preds:
    temp_score.append(pred['score'])
    temp_prediction.append(pred['token_str'])

    print(f">>> {pred['sequence']}")
    # print(pred)
  L_top_most_scores.append(temp_score)
  L_top_most_predictions.append(temp_prediction)
  print()

{'score': 0.08138512074947357, 'token': 5359, 'token_str': 'delivered', 'sequence': 'trump delivered the sad news on monday in the press briefing.'} 5
>>> trump delivered the sad news on monday in the press briefing.
>>> trump told the sad news on monday in the press briefing.
>>> trump said the sad news on monday in the press briefing.
>>> trump addressed the sad news on monday in the press briefing.
>>> trump described the sad news on monday in the press briefing.

{'score': 0.34503334760665894, 'token': 2170, 'token_str': 'called', 'sequence': 'trump called the elections after the results yesterday.'} 5
>>> trump called the elections after the results yesterday.
>>> trump won the elections after the results yesterday.
>>> trump canceled the elections after the results yesterday.
>>> trump lost the elections after the results yesterday.
>>> trump named the elections after the results yesterday.

{'score': 0.20589497685432434, 'token': 10173, 'token_str': 'predicted', 'sequence': 'bid

In [ ]:
R_top_most_scores = []
R_top_most_predictions = []
R_top_most_sentences = []
for prediction in validation_sentences_df['Masked Sentences']:
  preds = R_mask_filler(prediction)
  print(preds[0], len(preds))
  temp_score = []
  temp_prediction = []
  # R_top_most_scores.append(preds[0]['score'])
  # R_top_most_predictions.append(preds[0]['token_str'])
  R_top_most_sentences.append(preds[0]['sequence'])

  for pred in preds:
    temp_score.append(pred['score'])
    temp_prediction.append(pred['token_str'])

    print(f">>> {pred['sequence']}")
    # print(pred)
  R_top_most_scores.append(temp_score)
  R_top_most_predictions.append(temp_prediction)
  print()

{'score': 0.07356075942516327, 'token': 5359, 'token_str': 'delivered', 'sequence': 'trump delivered the sad news on monday in the press briefing.'} 5
>>> trump delivered the sad news on monday in the press briefing.
>>> trump told the sad news on monday in the press briefing.
>>> trump addressed the sad news on monday in the press briefing.
>>> trump described the sad news on monday in the press briefing.
>>> trump mentioned the sad news on monday in the press briefing.

{'score': 0.32672545313835144, 'token': 2170, 'token_str': 'called', 'sequence': 'trump called the elections after the results yesterday.'} 5
>>> trump called the elections after the results yesterday.
>>> trump canceled the elections after the results yesterday.
>>> trump won the elections after the results yesterday.
>>> trump cancelled the elections after the results yesterday.
>>> trump announced the elections after the results yesterday.

{'score': 0.2542664706707001, 'token': 10173, 'token_str': 'predicted', 'se

In [ ]:
validation_df = pd.DataFrame()
validation_df['L_mask_word_prediction'] = L_top_most_predictions
validation_df['L_sentence'] = L_top_most_sentences
validation_df['L_prediction_prob'] = L_top_most_scores
validation_df['R_mask_word_prediction'] = R_top_most_predictions
validation_df['R_sentence'] = R_top_most_sentences
validation_df['R_prediction_prob'] = R_top_most_scores

In [ ]:
validation_df

,L_mask_word_prediction,L_sentence,L_prediction_prob,R_mask_word_prediction,R_sentence,R_prediction_prob
0,"[delivered, told, said, addressed, described]",trump delivered the sad news on monday in the ...,"[0.08138512074947357, 0.06628639250993729, 0.0...","[delivered, told, addressed, described, mentio...",trump delivered the sad news on monday in the ...,"[0.07356075942516327, 0.06047539412975311, 0.0..."
1,"[called, won, canceled, lost, named]",trump called the elections after the results y...,"[0.34503334760665894, 0.08206091821193695, 0.0...","[called, canceled, won, cancelled, announced]",trump called the elections after the results y...,"[0.32672545313835144, 0.047891635447740555, 0...."
2,"[predicted, supported, praised, supports, endo...",biden predicted the victory of obama in 2012.,"[0.20589497685432434, 0.11232000589370728, 0.0...","[predicted, supported, praised, endorsed, supp...",biden predicted the victory of obama in 2012.,"[0.2542664706707001, 0.15217866003513336, 0.08..."
3,"[elected, reelected, fired, named, released]",trump was elected after the win of obama.,"[0.07329434156417847, 0.06654732674360275, 0.0...","[reelected, fired, elected, named, appointed]",trump was reelected after the win of obama.,"[0.06306461989879608, 0.06057732552289963, 0.0..."
4,"[##s, posted, used, has, released]","trumps tweets as us tackles coronavirus, inclu...","[0.18212540447711945, 0.06540976464748383, 0.0...","[posted, ##s, used, released, made]","trump posted tweets as us tackles coronavirus,...","[0.11232783645391464, 0.058183617889881134, 0...."
5,"[the, new, expanding, implementing, changing]",trump considering the coronavirus guidelines a...,"[0.11568285524845123, 0.07287382334470749, 0.0...","[the, implementing, adopting, expanding, new]",trump considering the coronavirus guidelines a...,"[0.09981061518192291, 0.06872966885566711, 0.0..."
6,"[the, new, changing, implementing, expanding]",biden considering the coronavirus guidelines a...,"[0.13971394300460815, 0.05805186554789543, 0.0...","[the, implementing, adopting, new, changing]",biden considering the coronavirus guidelines a...,"[0.11761776357889175, 0.05123515799641609, 0.0..."
7,"[leave, the, rebuild, reform, exit]",trump intends to leave country amid growing co...,"[0.30816951394081116, 0.046250008046627045, 0....","[leave, reform, exit, rebuild, abandon]",trump intends to leave country amid growing co...,"[0.33007100224494934, 0.09021633863449097, 0.0..."
8,"[the, this, another, a, his]",biden says he intendes to the country to preve...,"[0.6364176869392395, 0.0911860540509224, 0.072...","[the, another, this, his, a]",biden says he intendes to the country to preve...,"[0.46060705184936523, 0.18179969489574432, 0.1..."
9,"[blamed, criticized, sued, fired, supports]",california blamed trump for medical supply sho...,"[0.5015187859535217, 0.09938707947731018, 0.05...","[blamed, criticized, endorsed, supports, sued]",california blamed trump for medical supply sho...,"[0.49254390597343445, 0.10445214062929153, 0.0..."


In [ ]:
L_lexicon_word_valence_list = []
R_lexicon_word_valence_list = []
words_not_in_lexicon_dict = []

for words in L_top_most_predictions:
  avg_sum = 0
  for word in words:
    lemmatized_word = lemmatizer.lemmatize(word)
    stemmed_word = ps.stem(word)
    lemmatized_word = stemmed_word
    if lemmatized_word not in lexicon_dictionary:
      continue
    else:
      avg_sum += lexicon_dictionary[lemmatized_word]
  L_lexicon_word_valence_list.append(avg_sum/5)

R_lexicon_word_valence_list = []
for words in R_top_most_predictions:
  avg_sum = 0
  ranks = [1.4, 1.3, 1.2, 1.1, 1]
  for i, word in enumerate(words):
    lemmatized_word = lemmatizer.lemmatize(word)
    stemmed_word = ps.stem(word)
    lemmatized_word = stemmed_word
    if lemmatized_word not in lexicon_dictionary:
      continue
    else:
      avg_sum += (lexicon_dictionary[lemmatized_word] * ranks[i])
  R_lexicon_word_valence_list.append(avg_sum/5)

In [ ]:
validation_df['L_lexicon'] = L_lexicon_word_valence_list
validation_df['R_lexicon'] = R_lexicon_word_valence_list

In [ ]:
validation_df

,L_mask_word_prediction,L_sentence,L_prediction_prob,R_mask_word_prediction,R_sentence,R_prediction_prob,L_lexicon,R_lexicon
0,"[delivered, told, said, addressed, described]",trump delivered the sad news on monday in the ...,"[0.08138512074947357, 0.06628639250993729, 0.0...","[delivered, told, addressed, described, mentio...",trump delivered the sad news on monday in the ...,"[0.07356075942516327, 0.06047539412975311, 0.0...",0.3270,0.51728
1,"[called, won, canceled, lost, named]",trump called the elections after the results y...,"[0.34503334760665894, 0.08206091821193695, 0.0...","[called, canceled, won, cancelled, announced]",trump called the elections after the results y...,"[0.32672545313835144, 0.047891635447740555, 0....",0.3396,0.39816
2,"[predicted, supported, praised, supports, endo...",biden predicted the victory of obama in 2012.,"[0.20589497685432434, 0.11232000589370728, 0.0...","[predicted, supported, praised, endorsed, supp...",biden predicted the victory of obama in 2012.,"[0.2542664706707001, 0.15217866003513336, 0.08...",0.7938,0.93754
3,"[elected, reelected, fired, named, released]",trump was elected after the win of obama.,"[0.07329434156417847, 0.06654732674360275, 0.0...","[reelected, fired, elected, named, appointed]",trump was reelected after the win of obama.,"[0.06306461989879608, 0.06057732552289963, 0.0...",0.5704,0.64298
4,"[##s, posted, used, has, released]","trumps tweets as us tackles coronavirus, inclu...","[0.18212540447711945, 0.06540976464748383, 0.0...","[posted, ##s, used, released, made]","trump posted tweets as us tackles coronavirus,...","[0.11232783645391464, 0.058183617889881134, 0....",0.4340,0.52472
5,"[the, new, expanding, implementing, changing]",trump considering the coronavirus guidelines a...,"[0.11568285524845123, 0.07287382334470749, 0.0...","[the, implementing, adopting, expanding, new]",trump considering the coronavirus guidelines a...,"[0.09981061518192291, 0.06872966885566711, 0.0...",0.5582,0.62984
6,"[the, new, changing, implementing, expanding]",biden considering the coronavirus guidelines a...,"[0.13971394300460815, 0.05805186554789543, 0.0...","[the, implementing, adopting, new, changing]",biden considering the coronavirus guidelines a...,"[0.11761776357889175, 0.05123515799641609, 0.0...",0.5582,0.59164
7,"[leave, the, rebuild, reform, exit]",trump intends to leave country amid growing co...,"[0.30816951394081116, 0.046250008046627045, 0....","[leave, reform, exit, rebuild, abandon]",trump intends to leave country amid growing co...,"[0.33007100224494934, 0.09021633863449097, 0.0...",0.3502,0.45048
8,"[the, this, another, a, his]",biden says he intendes to the country to preve...,"[0.6364176869392395, 0.0911860540509224, 0.072...","[the, another, this, his, a]",biden says he intendes to the country to preve...,"[0.46060705184936523, 0.18179969489574432, 0.1...",0.0000,0.00000
9,"[blamed, criticized, sued, fired, supports]",california blamed trump for medical supply sho...,"[0.5015187859535217, 0.09938707947731018, 0.05...","[blamed, criticized, endorsed, supports, sued]",california blamed trump for medical supply sho...,"[0.49254390597343445, 0.10445214062929153, 0.0...",0.2836,0.48726


Validation scores for Method 5

In [ ]:
# Left leaning model (considering average) has lower score and right leaning model (considering rank average) has higher score
print(validation_df['L_lexicon'].sum()/50, validation_df['R_lexicon'].sum()/50)

0.4564399999999999 0.5526207999999999


### *Method 6 - Take RANK average of 5 for L-leaning and average of 5 for R-leaning*

In [ ]:
L_top_most_scores = []
L_top_most_predictions = []
L_top_most_sentences = []
for prediction in validation_sentences_df['Masked Sentences']:
  preds = L_mask_filler(prediction)
  print(preds[0], len(preds))
  temp_score = []
  temp_prediction = []
  # L_top_most_scores.append(preds[0]['score'])
  # L_top_most_predictions.append(preds[0]['token_str'])
  L_top_most_sentences.append(preds[0]['sequence'])

  for pred in preds:
    temp_score.append(pred['score'])
    temp_prediction.append(pred['token_str'])

    print(f">>> {pred['sequence']}")
    # print(pred)
  L_top_most_scores.append(temp_score)
  L_top_most_predictions.append(temp_prediction)
  print()

{'score': 0.08138512074947357, 'token': 5359, 'token_str': 'delivered', 'sequence': 'trump delivered the sad news on monday in the press briefing.'} 5
>>> trump delivered the sad news on monday in the press briefing.
>>> trump told the sad news on monday in the press briefing.
>>> trump said the sad news on monday in the press briefing.
>>> trump addressed the sad news on monday in the press briefing.
>>> trump described the sad news on monday in the press briefing.

{'score': 0.34503334760665894, 'token': 2170, 'token_str': 'called', 'sequence': 'trump called the elections after the results yesterday.'} 5
>>> trump called the elections after the results yesterday.
>>> trump won the elections after the results yesterday.
>>> trump canceled the elections after the results yesterday.
>>> trump lost the elections after the results yesterday.
>>> trump named the elections after the results yesterday.

{'score': 0.20589497685432434, 'token': 10173, 'token_str': 'predicted', 'sequence': 'bid

In [ ]:
R_top_most_scores = []
R_top_most_predictions = []
R_top_most_sentences = []
for prediction in validation_sentences_df['Masked Sentences']:
  preds = R_mask_filler(prediction)
  print(preds[0], len(preds))
  temp_score = []
  temp_prediction = []
  # R_top_most_scores.append(preds[0]['score'])
  # R_top_most_predictions.append(preds[0]['token_str'])
  R_top_most_sentences.append(preds[0]['sequence'])

  for pred in preds:
    temp_score.append(pred['score'])
    temp_prediction.append(pred['token_str'])

    print(f">>> {pred['sequence']}")
    # print(pred)
  R_top_most_scores.append(temp_score)
  R_top_most_predictions.append(temp_prediction)
  print()

{'score': 0.07356075942516327, 'token': 5359, 'token_str': 'delivered', 'sequence': 'trump delivered the sad news on monday in the press briefing.'} 5
>>> trump delivered the sad news on monday in the press briefing.
>>> trump told the sad news on monday in the press briefing.
>>> trump addressed the sad news on monday in the press briefing.
>>> trump described the sad news on monday in the press briefing.
>>> trump mentioned the sad news on monday in the press briefing.

{'score': 0.32672545313835144, 'token': 2170, 'token_str': 'called', 'sequence': 'trump called the elections after the results yesterday.'} 5
>>> trump called the elections after the results yesterday.
>>> trump canceled the elections after the results yesterday.
>>> trump won the elections after the results yesterday.
>>> trump cancelled the elections after the results yesterday.
>>> trump announced the elections after the results yesterday.

{'score': 0.2542664706707001, 'token': 10173, 'token_str': 'predicted', 'se

In [ ]:
validation_df = pd.DataFrame()
validation_df['L_mask_word_prediction'] = L_top_most_predictions
validation_df['L_sentence'] = L_top_most_sentences
validation_df['L_prediction_prob'] = L_top_most_scores
validation_df['R_mask_word_prediction'] = R_top_most_predictions
validation_df['R_sentence'] = R_top_most_sentences
validation_df['R_prediction_prob'] = R_top_most_scores

In [ ]:
validation_df

,L_mask_word_prediction,L_sentence,L_prediction_prob,R_mask_word_prediction,R_sentence,R_prediction_prob
0,"[delivered, told, said, addressed, described]",trump delivered the sad news on monday in the ...,"[0.08138512074947357, 0.06628639250993729, 0.0...","[delivered, told, addressed, described, mentio...",trump delivered the sad news on monday in the ...,"[0.07356075942516327, 0.06047539412975311, 0.0..."
1,"[called, won, canceled, lost, named]",trump called the elections after the results y...,"[0.34503334760665894, 0.08206091821193695, 0.0...","[called, canceled, won, cancelled, announced]",trump called the elections after the results y...,"[0.32672545313835144, 0.047891635447740555, 0...."
2,"[predicted, supported, praised, supports, endo...",biden predicted the victory of obama in 2012.,"[0.20589497685432434, 0.11232000589370728, 0.0...","[predicted, supported, praised, endorsed, supp...",biden predicted the victory of obama in 2012.,"[0.2542664706707001, 0.15217866003513336, 0.08..."
3,"[elected, reelected, fired, named, released]",trump was elected after the win of obama.,"[0.07329434156417847, 0.06654732674360275, 0.0...","[reelected, fired, elected, named, appointed]",trump was reelected after the win of obama.,"[0.06306461989879608, 0.06057732552289963, 0.0..."
4,"[##s, posted, used, has, released]","trumps tweets as us tackles coronavirus, inclu...","[0.18212540447711945, 0.06540976464748383, 0.0...","[posted, ##s, used, released, made]","trump posted tweets as us tackles coronavirus,...","[0.11232783645391464, 0.058183617889881134, 0...."
5,"[the, new, expanding, implementing, changing]",trump considering the coronavirus guidelines a...,"[0.11568285524845123, 0.07287382334470749, 0.0...","[the, implementing, adopting, expanding, new]",trump considering the coronavirus guidelines a...,"[0.09981061518192291, 0.06872966885566711, 0.0..."
6,"[the, new, changing, implementing, expanding]",biden considering the coronavirus guidelines a...,"[0.13971394300460815, 0.05805186554789543, 0.0...","[the, implementing, adopting, new, changing]",biden considering the coronavirus guidelines a...,"[0.11761776357889175, 0.05123515799641609, 0.0..."
7,"[leave, the, rebuild, reform, exit]",trump intends to leave country amid growing co...,"[0.30816951394081116, 0.046250008046627045, 0....","[leave, reform, exit, rebuild, abandon]",trump intends to leave country amid growing co...,"[0.33007100224494934, 0.09021633863449097, 0.0..."
8,"[the, this, another, a, his]",biden says he intendes to the country to preve...,"[0.6364176869392395, 0.0911860540509224, 0.072...","[the, another, this, his, a]",biden says he intendes to the country to preve...,"[0.46060705184936523, 0.18179969489574432, 0.1..."
9,"[blamed, criticized, sued, fired, supports]",california blamed trump for medical supply sho...,"[0.5015187859535217, 0.09938707947731018, 0.05...","[blamed, criticized, endorsed, supports, sued]",california blamed trump for medical supply sho...,"[0.49254390597343445, 0.10445214062929153, 0.0..."


In [ ]:
L_lexicon_word_valence_list = []
R_lexicon_word_valence_list = []
words_not_in_lexicon_dict = []

for words in R_top_most_predictions:
  avg_sum = 0
  for word in words:
    lemmatized_word = lemmatizer.lemmatize(word)
    stemmed_word = ps.stem(word)
    lemmatized_word = stemmed_word
    if lemmatized_word not in lexicon_dictionary:
      continue
    else:
      avg_sum += lexicon_dictionary[lemmatized_word]
  R_lexicon_word_valence_list.append(avg_sum/5)

for words in L_top_most_predictions:
  avg_sum = 0
  ranks = [1.4, 1.3, 1.2, 1.1, 1]
  for i, word in enumerate(words):
    lemmatized_word = lemmatizer.lemmatize(word)
    stemmed_word = ps.stem(word)
    lemmatized_word = stemmed_word
    if lemmatized_word not in lexicon_dictionary:
      continue
    else:
      avg_sum += (lexicon_dictionary[lemmatized_word] * ranks[i])
  L_lexicon_word_valence_list.append(avg_sum/5)

In [ ]:
validation_df['L_lexicon'] = L_lexicon_word_valence_list
validation_df['R_lexicon'] = R_lexicon_word_valence_list

In [ ]:
validation_df

,L_mask_word_prediction,L_sentence,L_prediction_prob,R_mask_word_prediction,R_sentence,R_prediction_prob,L_lexicon,R_lexicon
0,"[delivered, told, said, addressed, described]",trump delivered the sad news on monday in the ...,"[0.08138512074947357, 0.06628639250993729, 0.0...","[delivered, told, addressed, described, mentio...",trump delivered the sad news on monday in the ...,"[0.07356075942516327, 0.06047539412975311, 0.0...",0.38384,0.4394
1,"[called, won, canceled, lost, named]",trump called the elections after the results y...,"[0.34503334760665894, 0.08206091821193695, 0.0...","[called, canceled, won, cancelled, announced]",trump called the elections after the results y...,"[0.32672545313835144, 0.047891635447740555, 0....",0.40118,0.3316
2,"[predicted, supported, praised, supports, endo...",biden predicted the victory of obama in 2012.,"[0.20589497685432434, 0.11232000589370728, 0.0...","[predicted, supported, praised, endorsed, supp...",biden predicted the victory of obama in 2012.,"[0.2542664706707001, 0.15217866003513336, 0.08...",0.93836,0.7938
3,"[elected, reelected, fired, named, released]",trump was elected after the win of obama.,"[0.07329434156417847, 0.06654732674360275, 0.0...","[reelected, fired, elected, named, appointed]",trump was reelected after the win of obama.,"[0.06306461989879608, 0.06057732552289963, 0.0...",0.67268,0.5506
4,"[##s, posted, used, has, released]","trumps tweets as us tackles coronavirus, inclu...","[0.18212540447711945, 0.06540976464748383, 0.0...","[posted, ##s, used, released, made]","trump posted tweets as us tackles coronavirus,...","[0.11232783645391464, 0.058183617889881134, 0....",0.49744,0.4340
5,"[the, new, expanding, implementing, changing]",trump considering the coronavirus guidelines a...,"[0.11568285524845123, 0.07287382334470749, 0.0...","[the, implementing, adopting, expanding, new]",trump considering the coronavirus guidelines a...,"[0.09981061518192291, 0.06872966885566711, 0.0...",0.64740,0.5502
6,"[the, new, changing, implementing, expanding]",biden considering the coronavirus guidelines a...,"[0.13971394300460815, 0.05805186554789543, 0.0...","[the, implementing, adopting, new, changing]",biden considering the coronavirus guidelines a...,"[0.11761776357889175, 0.05123515799641609, 0.0...",0.63984,0.5124
7,"[leave, the, rebuild, reform, exit]",trump intends to leave country amid growing co...,"[0.30816951394081116, 0.046250008046627045, 0....","[leave, reform, exit, rebuild, abandon]",trump intends to leave country amid growing co...,"[0.33007100224494934, 0.09021633863449097, 0.0...",0.40072,0.3758
8,"[the, this, another, a, his]",biden says he intendes to the country to preve...,"[0.6364176869392395, 0.0911860540509224, 0.072...","[the, another, this, his, a]",biden says he intendes to the country to preve...,"[0.46060705184936523, 0.18179969489574432, 0.1...",0.00000,0.0000
9,"[blamed, criticized, sued, fired, supports]",california blamed trump for medical supply sho...,"[0.5015187859535217, 0.09938707947731018, 0.05...","[blamed, criticized, endorsed, supports, sued]",california blamed trump for medical supply sho...,"[0.49254390597343445, 0.10445214062929153, 0.0...",0.31096,0.4128


Validation scores for Method 6

In [ ]:
# Left leaning model (considering RANK average) has higher score and right leaning model (considering average) has higher score
print(validation_df['L_lexicon'].sum()/50, validation_df['R_lexicon'].sum()/50)

0.5405556 0.46281999999999995


### *Method 7 - Take RANK average of 5 for L-leaning and RANK average of 5 for R-leaning*

In [ ]:
## TESTING
import numpy as np
L_top_most_scores = []
L_top_most_predictions = []
L_top_most_sentences = []

## TESTING

for prediction in validation_sentences_df['Masked Sentences']:
  inputs = tokenizer(prediction, return_tensors="np")
  token_logits = left_leaning_model(**inputs).logits
  # Find the location of [MASK] and extract its logits
  mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
  mask_token_logits = token_logits[0, mask_token_index, :]
  # Pick the [MASK] candidates with the highest logits
  # We negate the array before argsort to get the largest, not the smallest, logits
  top_5_tokens = np.argsort(-mask_token_logits)[:5].tolist()

  for token in top_5_tokens:
    print(f">>> {prediction.replace(tokenizer.mask_token, tokenizer.decode([token]))}")
  # all_tokens = np.argsort(-mask_token_logits).tolist()
  print()


>>> Trump delivered the sad news on Monday in the press briefing.
>>> Trump told the sad news on Monday in the press briefing.
>>> Trump said the sad news on Monday in the press briefing.
>>> Trump shared the sad news on Monday in the press briefing.
>>> Trump made the sad news on Monday in the press briefing.

>>> Trump called the elections after the results yesterday.
>>> Trump won the elections after the results yesterday.
>>> Trump lost the elections after the results yesterday.
>>> Trump entered the elections after the results yesterday.
>>> Trump canceled the elections after the results yesterday.

>>> Biden predicted the victory of Obama in 2012.
>>> Biden praised the victory of Obama in 2012.
>>> Biden supported the victory of Obama in 2012.
>>> Biden celebrated the victory of Obama in 2012.
>>> Biden after the victory of Obama in 2012.

>>> Trump was reelected after the win of Obama.
>>> Trump was elected after the win of Obama.
>>> Trump was fired after the win of Obama.
>>> 

In [ ]:
import numpy as np

## TESTING

for prediction in validation_sentences_df['Masked Sentences']:
  inputs = tokenizer(prediction, return_tensors="np")
  token_logits = left_leaning_model(**inputs).logits
  # Find the location of [MASK] and extract its logits
  mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
  mask_token_logits = token_logits[0, mask_token_index, :]
  # Pick the [MASK] candidates with the highest logits
  # We negate the array before argsort to get the largest, not the smallest, logits
  top_5_tokens = np.argsort(-mask_token_logits)[:5].tolist()

  for token in top_5_tokens:
    print(f">>> {prediction.replace(tokenizer.mask_token, tokenizer.decode([token]))}")

  print()

>>> Republicans shared the sad news on Monday in the press briefing.
>>> Republicans broke the sad news on Monday in the press briefing.
>>> Republicans delivered the sad news on Monday in the press briefing.
>>> Republicans welcomed the sad news on Monday in the press briefing.
>>> Republicans repeated the sad news on Monday in the press briefing.

>>> Republicans won the elections after the results yesterday.
>>> Republicans entered the elections after the results yesterday.
>>> Republicans lost the elections after the results yesterday.
>>> Republicans called the elections after the results yesterday.
>>> Republicans dominated the elections after the results yesterday.

>>> Democrats support the victory of Obama in 2012.
>>> Democrats supported the victory of Obama in 2012.
>>> Democrats recognize the victory of Obama in 2012.
>>> Democrats predicted the victory of Obama in 2012.
>>> Democrats praised the victory of Obama in 2012.

>>> Republicans was formed after the win of Obama.


In [ ]:
L_top_most_scores = []
L_top_most_predictions = []
L_top_most_sentences = []
for prediction in validation_sentences_df['Masked Sentences']:
  preds = L_mask_filler(prediction)
  print(preds[0], len(preds))
  temp_score = []
  temp_prediction = []
  # L_top_most_scores.append(preds[0]['score'])
  # L_top_most_predictions.append(preds[0]['token_str'])
  L_top_most_sentences.append(preds[0]['sequence'])

  for pred in preds:
    temp_score.append(pred['score'])
    temp_prediction.append(pred['token_str'])

    print(f">>> {pred['sequence']}")
    # print(pred)
  L_top_most_scores.append(temp_score)
  L_top_most_predictions.append(temp_prediction)
  print()

{'score': 0.08138512074947357, 'token': 5359, 'token_str': 'delivered', 'sequence': 'trump delivered the sad news on monday in the press briefing.'} 5
>>> trump delivered the sad news on monday in the press briefing.
>>> trump told the sad news on monday in the press briefing.
>>> trump said the sad news on monday in the press briefing.
>>> trump addressed the sad news on monday in the press briefing.
>>> trump described the sad news on monday in the press briefing.

{'score': 0.34503334760665894, 'token': 2170, 'token_str': 'called', 'sequence': 'trump called the elections after the results yesterday.'} 5
>>> trump called the elections after the results yesterday.
>>> trump won the elections after the results yesterday.
>>> trump canceled the elections after the results yesterday.
>>> trump lost the elections after the results yesterday.
>>> trump named the elections after the results yesterday.

{'score': 0.20589497685432434, 'token': 10173, 'token_str': 'predicted', 'sequence': 'bid

In [ ]:
R_top_most_scores = []
R_top_most_predictions = []
R_top_most_sentences = []
for prediction in validation_sentences_df['Masked Sentences']:
  preds = R_mask_filler(prediction)
  print(preds[0], len(preds))
  temp_score = []
  temp_prediction = []
  # R_top_most_scores.append(preds[0]['score'])
  # R_top_most_predictions.append(preds[0]['token_str'])
  R_top_most_sentences.append(preds[0]['sequence'])

  for pred in preds:
    temp_score.append(pred['score'])
    temp_prediction.append(pred['token_str'])

    print(f">>> {pred['sequence']}")
    # print(pred)
  R_top_most_scores.append(temp_score)
  R_top_most_predictions.append(temp_prediction)
  print()

{'score': 0.07356075942516327, 'token': 5359, 'token_str': 'delivered', 'sequence': 'trump delivered the sad news on monday in the press briefing.'} 5
>>> trump delivered the sad news on monday in the press briefing.
>>> trump told the sad news on monday in the press briefing.
>>> trump addressed the sad news on monday in the press briefing.
>>> trump described the sad news on monday in the press briefing.
>>> trump mentioned the sad news on monday in the press briefing.

{'score': 0.32672545313835144, 'token': 2170, 'token_str': 'called', 'sequence': 'trump called the elections after the results yesterday.'} 5
>>> trump called the elections after the results yesterday.
>>> trump canceled the elections after the results yesterday.
>>> trump won the elections after the results yesterday.
>>> trump cancelled the elections after the results yesterday.
>>> trump announced the elections after the results yesterday.

{'score': 0.2542664706707001, 'token': 10173, 'token_str': 'predicted', 'se

In [ ]:
validation_df = pd.DataFrame()
validation_df['L_mask_word_prediction'] = L_top_most_predictions
validation_df['L_sentence'] = L_top_most_sentences
validation_df['L_prediction_prob'] = L_top_most_scores
validation_df['R_mask_word_prediction'] = R_top_most_predictions
validation_df['R_sentence'] = R_top_most_sentences
validation_df['R_prediction_prob'] = R_top_most_scores

In [ ]:
validation_df

,L_mask_word_prediction,L_sentence,L_prediction_prob,R_mask_word_prediction,R_sentence,R_prediction_prob
0,"[delivered, told, said, addressed, described]",trump delivered the sad news on monday in the ...,"[0.08138512074947357, 0.06628639250993729, 0.0...","[delivered, told, addressed, described, mentio...",trump delivered the sad news on monday in the ...,"[0.07356075942516327, 0.06047539412975311, 0.0..."
1,"[called, won, canceled, lost, named]",trump called the elections after the results y...,"[0.34503334760665894, 0.08206091821193695, 0.0...","[called, canceled, won, cancelled, announced]",trump called the elections after the results y...,"[0.32672545313835144, 0.047891635447740555, 0...."
2,"[predicted, supported, praised, supports, endo...",biden predicted the victory of obama in 2012.,"[0.20589497685432434, 0.11232000589370728, 0.0...","[predicted, supported, praised, endorsed, supp...",biden predicted the victory of obama in 2012.,"[0.2542664706707001, 0.15217866003513336, 0.08..."
3,"[elected, reelected, fired, named, released]",trump was elected after the win of obama.,"[0.07329434156417847, 0.06654732674360275, 0.0...","[reelected, fired, elected, named, appointed]",trump was reelected after the win of obama.,"[0.06306461989879608, 0.06057732552289963, 0.0..."
4,"[##s, posted, used, has, released]","trumps tweets as us tackles coronavirus, inclu...","[0.18212540447711945, 0.06540976464748383, 0.0...","[posted, ##s, used, released, made]","trump posted tweets as us tackles coronavirus,...","[0.11232783645391464, 0.058183617889881134, 0...."
5,"[the, new, expanding, implementing, changing]",trump considering the coronavirus guidelines a...,"[0.11568285524845123, 0.07287382334470749, 0.0...","[the, implementing, adopting, expanding, new]",trump considering the coronavirus guidelines a...,"[0.09981061518192291, 0.06872966885566711, 0.0..."
6,"[the, new, changing, implementing, expanding]",biden considering the coronavirus guidelines a...,"[0.13971394300460815, 0.05805186554789543, 0.0...","[the, implementing, adopting, new, changing]",biden considering the coronavirus guidelines a...,"[0.11761776357889175, 0.05123515799641609, 0.0..."
7,"[leave, the, rebuild, reform, exit]",trump intends to leave country amid growing co...,"[0.30816951394081116, 0.046250008046627045, 0....","[leave, reform, exit, rebuild, abandon]",trump intends to leave country amid growing co...,"[0.33007100224494934, 0.09021633863449097, 0.0..."
8,"[the, this, another, a, his]",biden says he intendes to the country to preve...,"[0.6364176869392395, 0.0911860540509224, 0.072...","[the, another, this, his, a]",biden says he intendes to the country to preve...,"[0.46060705184936523, 0.18179969489574432, 0.1..."
9,"[blamed, criticized, sued, fired, supports]",california blamed trump for medical supply sho...,"[0.5015187859535217, 0.09938707947731018, 0.05...","[blamed, criticized, endorsed, supports, sued]",california blamed trump for medical supply sho...,"[0.49254390597343445, 0.10445214062929153, 0.0..."


In [ ]:
L_lexicon_word_valence_list = []
R_lexicon_word_valence_list = []
words_not_in_lexicon_dict = []

for words in L_top_most_predictions:
  avg_sum = 0
  ranks = [1.4, 1.3, 1.2, 1.1, 1]
  for i, word in enumerate(words):
    lemmatized_word = lemmatizer.lemmatize(word)
    stemmed_word = ps.stem(word)
    lemmatized_word = stemmed_word
    if lemmatized_word not in lexicon_dictionary:
      continue
    else:
      avg_sum += (lexicon_dictionary[lemmatized_word] * ranks[i])
  L_lexicon_word_valence_list.append(avg_sum/5)

for words in R_top_most_predictions:
  avg_sum = 0
  ranks = [1.4, 1.3, 1.2, 1.1, 1]
  for i, word in enumerate(words):
    lemmatized_word = lemmatizer.lemmatize(word)
    stemmed_word = ps.stem(word)
    lemmatized_word = stemmed_word
    if lemmatized_word not in lexicon_dictionary:
      continue
    else:
      avg_sum += (lexicon_dictionary[lemmatized_word] * ranks[i])
  R_lexicon_word_valence_list.append(avg_sum/5)

In [ ]:
validation_df['L_lexicon'] = L_lexicon_word_valence_list
validation_df['R_lexicon'] = R_lexicon_word_valence_list

In [ ]:
validation_df

,L_mask_word_prediction,L_sentence,L_prediction_prob,R_mask_word_prediction,R_sentence,R_prediction_prob,L_lexicon,R_lexicon
0,"[delivered, told, said, addressed, described]",trump delivered the sad news on monday in the ...,"[0.08138512074947357, 0.06628639250993729, 0.0...","[delivered, told, addressed, described, mentio...",trump delivered the sad news on monday in the ...,"[0.07356075942516327, 0.06047539412975311, 0.0...",0.38384,0.51728
1,"[called, won, canceled, lost, named]",trump called the elections after the results y...,"[0.34503334760665894, 0.08206091821193695, 0.0...","[called, canceled, won, cancelled, announced]",trump called the elections after the results y...,"[0.32672545313835144, 0.047891635447740555, 0....",0.40118,0.39816
2,"[predicted, supported, praised, supports, endo...",biden predicted the victory of obama in 2012.,"[0.20589497685432434, 0.11232000589370728, 0.0...","[predicted, supported, praised, endorsed, supp...",biden predicted the victory of obama in 2012.,"[0.2542664706707001, 0.15217866003513336, 0.08...",0.93836,0.93754
3,"[elected, reelected, fired, named, released]",trump was elected after the win of obama.,"[0.07329434156417847, 0.06654732674360275, 0.0...","[reelected, fired, elected, named, appointed]",trump was reelected after the win of obama.,"[0.06306461989879608, 0.06057732552289963, 0.0...",0.67268,0.64298
4,"[##s, posted, used, has, released]","trumps tweets as us tackles coronavirus, inclu...","[0.18212540447711945, 0.06540976464748383, 0.0...","[posted, ##s, used, released, made]","trump posted tweets as us tackles coronavirus,...","[0.11232783645391464, 0.058183617889881134, 0....",0.49744,0.52472
5,"[the, new, expanding, implementing, changing]",trump considering the coronavirus guidelines a...,"[0.11568285524845123, 0.07287382334470749, 0.0...","[the, implementing, adopting, expanding, new]",trump considering the coronavirus guidelines a...,"[0.09981061518192291, 0.06872966885566711, 0.0...",0.64740,0.62984
6,"[the, new, changing, implementing, expanding]",biden considering the coronavirus guidelines a...,"[0.13971394300460815, 0.05805186554789543, 0.0...","[the, implementing, adopting, new, changing]",biden considering the coronavirus guidelines a...,"[0.11761776357889175, 0.05123515799641609, 0.0...",0.63984,0.59164
7,"[leave, the, rebuild, reform, exit]",trump intends to leave country amid growing co...,"[0.30816951394081116, 0.046250008046627045, 0....","[leave, reform, exit, rebuild, abandon]",trump intends to leave country amid growing co...,"[0.33007100224494934, 0.09021633863449097, 0.0...",0.40072,0.45048
8,"[the, this, another, a, his]",biden says he intendes to the country to preve...,"[0.6364176869392395, 0.0911860540509224, 0.072...","[the, another, this, his, a]",biden says he intendes to the country to preve...,"[0.46060705184936523, 0.18179969489574432, 0.1...",0.00000,0.00000
9,"[blamed, criticized, sued, fired, supports]",california blamed trump for medical supply sho...,"[0.5015187859535217, 0.09938707947731018, 0.05...","[blamed, criticized, endorsed, supports, sued]",california blamed trump for medical supply sho...,"[0.49254390597343445, 0.10445214062929153, 0.0...",0.31096,0.48726


Validation scores for Method 7

In [ ]:
# Left leaning model (considering RANK average) has lower score and right leaning model (considering RANK average) has slightly higher score
print(validation_df['L_lexicon'].sum()/50, validation_df['R_lexicon'].sum()/50)

0.5405556 0.5526207999999999


### Method 8 - Checking scores for "democrats" and "republicans"

In [ ]:
import numpy as np
L_democrats_prob_score = []
L_republicans_prob_score = []
L_trump_prob_score = []

## TESTING

for prediction in validation_sentences_df['Target Masked Sentences']:
  inputs = tokenizer(prediction, return_tensors="np")
  token_logits = left_leaning_model(**inputs).logits
  # Find the location of [MASK] and extract its logits
  mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
  mask_token_logits = token_logits[0, mask_token_index, :]
  # Pick the [MASK] candidates with the highest logits
  # We negate the array before argsort to get the largest, not the smallest, logits
  all_tokens = np.argsort(-mask_token_logits).tolist()

  word_probability_dict = {}
  for index, val in enumerate(mask_token_logits):
    word = tokenizer.decode([index])
    word_probability_dict[word] = val.numpy()
  print('Prediction:', prediction)
  print("Democrats prob. score:", word_probability_dict['democrats'])
  print("Republicans prob. score:", word_probability_dict['republicans'])
  print("Trump prob. score:", word_probability_dict['trump'])
  print()
  L_democrats_prob_score.append(word_probability_dict['democrats'])
  L_republicans_prob_score.append(word_probability_dict['republicans'])
  L_trump_prob_score.append(word_probability_dict['trump'])



Prediction: [MASK] announced the sad news on Monday in the press briefing.
Democrats prob. score: 2.707
Republicans prob. score: 2.41
Trump prob. score: 7.035

Prediction: [MASK] won the elections after the results yesterday. 
Democrats prob. score: 5.188
Republicans prob. score: 5.19
Trump prob. score: 5.254

Prediction: [MASK] celebrated the victory of Obama in 2012.
Democrats prob. score: 3.035
Republicans prob. score: 2.74
Trump prob. score: 5.066

Prediction: [MASK] was dissappointed after the win of Obama
Democrats prob. score: 1.718
Republicans prob. score: 1.511
Trump prob. score: 6.953

Prediction: [MASK] fires-off tweets as US tackles coronavirus, including messages supporting rollback of guidelines to mitigate spread
Democrats prob. score: 0.1963
Republicans prob. score: 0.005127
Trump prob. score: 4.043

Prediction: [MASK] considering loosening coronavirus guidelines amid growing economic fallout.
Democrats prob. score: 3.848
Republicans prob. score: 3.041
Trump prob. score

In [ ]:
print(len(L_republicans_prob_score))
print(len(L_trump_prob_score))

57
57


In [ ]:
validation_df = pd.DataFrame()
validation_df['org_sentence'] = validation_sentences_df['Target Masked Sentences']
validation_df['L_democrats_prob_score'] = L_democrats_prob_score
validation_df['L_republicans_prob_score'] = L_republicans_prob_score
validation_df['L_trump_prob_score'] = L_trump_prob_score
print(validation_df)


                                         org_sentence  L_democrats_prob_score  \
0   [MASK] announced the sad news on Monday in the...                2.707031   
1   [MASK] won the elections after the results yes...                5.187500   
2     [MASK] celebrated the victory of Obama in 2012.                3.035156   
3     [MASK] was dissappointed after the win of Obama                1.717773   
4   [MASK] fires-off tweets as US tackles coronavi...                0.196289   
5   [MASK] considering loosening coronavirus guide...                3.847656   
6   [MASK] considering strengthening coronavirus g...                3.800781   
7   [MASK] intends to open country amid growing co...                2.234375   
8   [MASK] says he intends to lockdown country to ...                0.971680   
9   California blames [MASK] for medical supply sh...                5.496094   
10  California supports [MASK] irrespective of ele...                6.960938   
11  [MASK] says it's time fo

In [ ]:
validation_df.to_csv (r'export_validation_df.csv', index = False, header=True)

### Method 9 - Target and Prior probability

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import numpy as np
L_target_democrats = []
L_target_republicans = []
L_target_trump = []

## TESTING

for prediction in validation_sentences_df['Target Masked Sentences']:
  inputs = tokenizer(prediction, return_tensors="np")
  token_logits = left_leaning_model(**inputs).logits
  # Find the location of [MASK] and extract its logits
  mask_token_index = np.argwhere(inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
  mask_token_logits = token_logits[0, mask_token_index, :]
  # Pick the [MASK] candidates with the highest logits
  # We negate the array before argsort to get the largest, not the smallest, logits
  all_tokens = np.argsort(-mask_token_logits).tolist()

  word_probability_dict = {}
  for index, val in enumerate(mask_token_logits):
    word = tokenizer.decode([index])
    word_probability_dict[word] = val.numpy()
  print('Prediction:', prediction)
  print("Democrats prob. score:", word_probability_dict['democrats'])
  print("Republicans prob. score:", word_probability_dict['republicans'])
  print("Trump prob. score:", word_probability_dict['trump'])
  print()
  L_target_democrats.append(word_probability_dict['democrats'])
  L_target_republicans.append(word_probability_dict['republicans'])
  L_target_trump.append(word_probability_dict['trump'])

L_prior_prob = []
for prediction in validation_sentences_df['Masked Sentences']:
  print(prediction)
  dem_prediction = ""
  rep_prediction = ""
  if "republicans" in prediction.lower():
    dem_prediction = prediction.lower().replace("republicans", "democrats").replace('[mask]', '[MASK]')
    rep_prediction = prediction.lower().replace('[mask]', '[MASK]')
  else:
    dem_prediction = prediction.lower().replace('[mask]', '[MASK]')
    rep_prediction = prediction.lower().replace("democrats", "republicans").replace('[mask]', '[MASK]')
  print(rep_prediction)
  print(dem_prediction)
  dem_inputs = tokenizer(dem_prediction, return_tensors="np")
  rep_inputs = tokenizer(rep_prediction, return_tensors="np")
  dem_token_logits = left_leaning_model(**inputs).logits
  rep_token_logits = left_leaning_model(**inputs).logits
  # Find the location of [MASK] and extract its logits
  dem_mask_token_index = np.argwhere(dem_inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
  dem_mask_token_logits = dem_token_logits[0, dem_mask_token_index, :]

  rep_mask_token_index = np.argwhere(rep_inputs["input_ids"] == tokenizer.mask_token_id)[0, 1]
  rep_mask_token_logits = rep_token_logits[0, rep_mask_token_index, :]
  # Pick the [MASK] candidates with the highest logits
  # We negate the array before argsort to get the largest, not the smallest, logits
  dem_all_tokens = np.argsort(-dem_mask_token_logits).tolist()
  rep_all_tokens = np.argsort(-rep_mask_token_logits).tolist()

  dem_word_probability_dict = {}
  for index, val in enumerate(dem_mask_token_logits):
    word = tokenizer.decode([index])
    dem_word_probability_dict[word] = val.numpy()
  rep_word_probability_dict = {}
  for index, val in enumerate(rep_mask_token_logits):
    word = tokenizer.decode([index])
    rep_word_probability_dict[word] = val.numpy()
  average_prob = (dem_word_probability_dict['democrats'] + rep_word_probability_dict['republicans']) / 2
  L_prior_prob.append(abs(average_prob))


Prediction: [MASK] announced the sad news on Monday in the press briefing.
Democrats prob. score: 2.7011998
Republicans prob. score: 2.4063332
Trump prob. score: 7.0456924

Prediction: [MASK] won the elections after the results yesterday. 
Democrats prob. score: 5.192281
Republicans prob. score: 5.192088
Trump prob. score: 5.2530518

Prediction: [MASK] celebrated the victory of Obama in 2012.
Democrats prob. score: 3.0291867
Republicans prob. score: 2.7362802
Trump prob. score: 5.063363

Prediction: [MASK] was dissappointed after the win of Obama
Democrats prob. score: 1.7138113
Republicans prob. score: 1.5072895
Trump prob. score: 6.953195

Prediction: [MASK] fires-off tweets as US tackles coronavirus, including messages supporting rollback of guidelines to mitigate spread
Democrats prob. score: 0.19503543
Republicans prob. score: 0.0051742047
Trump prob. score: 4.0401745

Prediction: [MASK] considering loosening coronavirus guidelines amid growing economic fallout.
Democrats prob. sc

In [ ]:
L_target_democrats

[2.7011998,
 5.192281,
 3.0291867,
 1.7138113,
 0.19503543,
 3.8519275,
 3.8011024,
 2.23168,
 0.9644824,
 5.497117,
 6.9633946,
 6.4342318,
 6.782938,
 3.7518032,
 3.5766995,
 2.1497447,
 2.0196903,
 3.8519275,
 3.8011024,
 1.7130221,
 1.0155296,
 0.038226336,
 5.718599,
 2.1317198,
 0.22205076,
 2.398108,
 2.4673798,
 -1.9139705,
 -1.9139705,
 -1.6966507,
 0.7067963,
 0.46099827,
 3.3692615,
 3.1141715,
 3.6337144,
 2.8247983,
 4.1801424,
 -1.406319,
 6.3826427,
 7.19612,
 4.965305,
 6.0440555,
 5.127833,
 6.8969235,
 4.659451,
 4.894062,
 4.029246,
 1.4359561,
 8.48298,
 8.571899,
 6.4534264,
 8.875857,
 1.529835,
 -1.5568973,
 -0.44865194]

In [ ]:
validation_df = pd.DataFrame()
validation_df['org_sentence'] = validation_sentences_df['Actual Sentences']
validation_df['L_target_democrats'] = L_target_democrats
validation_df['L_target_republicans'] = L_target_republicans
validation_df['L_target_trump'] = L_target_trump
validation_df['L_prior'] = L_prior_prob


In [ ]:
validation_df.to_csv (r'L_validation_results_target_prior.csv', index = False, header=True)

# **Conclusion** - Right leaning model is slightly more positive than Left leaning model